In [ ]:
sc

In [ ]:
sc._conf.getAll()

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, FloatType, StringType, StructType, BooleanType, ArrayType, DoubleType
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF, VectorAssembler
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml import PipelineModel, pipeline
import math
import numpy as np
import name_tools
import jellyfish
import fuzzywuzzy
import phonetics
from name_tools import split, canonicalize
from jellyfish import jaro_winkler
import string
import re
from fuzzywuzzy import fuzz
import itertools
import geopy
from geopy import distance
from jellyfish import levenshtein_distance as ld
import json
from functools import reduce
import pandas as pd

In [ ]:
test = {"full_name":"Kazuyoshi Kato","linkedin_url":"http://www.linkedin.com/in/kazuyoshi","headline":"Open Source Software Developer","industry":"Retail","location":"Seattle, Washington","education":[{"name":"Toyo University","major":"Computer Science","summary":None}],"experience":[{"title":"Software Development Enginner","description":None,"organization":"Amazon"},{"title":"Software Development Engineer","description":None,"organization":"Amazon Japan"},{"title":"Engineer","description":"Developed \"mixi Platform\", a browser-based application platform and API for 3rd party developers. Our platform used by Zynga Japan, DeNA, HootSuite, many companies and many individual developers.\n http://developer.mixi.co.jp/en/\n\n Started and maintained their Jenkins based Continuos Integration system.\n http://alpha.mixi.co.jp/author/kzys/ (Japanese)","organization":"mixi, Inc"},{"title":"Technical staff (contract worker)","description":"Developed Wedata.\n http://wedata.net/\n http://www.wikisym.org/ws2012/bin/download/Main/Program/wikisym12submission53.pdf\n http://kaigi.org/jsai/webprogram/2011/pdf/487.pdf (Japanese)","organization":"AIST"},{"title":"Development engineer (part-time)","description":"Developed ActionScript support for VCE, their RPC framework.","organization":"Community Engine Inc."}],"interests":[],"summary":"I'm passionate about open source software and the web.","projects":[],"publications":[],"skills":["Scala","JavaScript","Objective-C","Perl","Ruby","Git","Linux"],"websites":[{"url":"http://8-p.info/","description":"Personal Website"}],"linkedin":213964086}

In [ ]:
git = [{"match":1,"linkedin_id":213964086,"gid":17517,"git_name":"Christian Berkhoff","git_login":"cberkhoff","git_location":"Seattle, WA","git_company":"Amazon LLC","git_org":[],"git_email":"christian.berkhoff@gmail.com","git_blog":None,"git_websiteUrl":None,"github_url":"https://github.com/cberkhoff","bio":None,"repos":[{"name":"howishell","url":"https://github.com/imella/howishell","description":"How I shell NASA Project","lang":["CoffeeScript","JavaScript"],"readme":"# How I Shell NASA Project\n\n* Daniel Aguilar\n* Christian Berkhoff\n* Joaquín Jaramillo\n* Ignacio Mella\n* Sebastían Rokas\n\n## Instalación\n\nEl único requisito es _node.js_\n\n    npm -g install handlebars coffee-script\n\n## Compilación\n\n    make\n\nEste comando compila `coffe/*.coffee` a `js/app.js` y `views\\*.hb` a `js/templates.js`\n\n# Moon Harvest\n\n## Overview\n\n### Do you have what's it takes to establish a colony on the Moon? What do you think it's best for the Moon?\n\nMan have always been fascinated with our natural satellite and there are a number of reasons to create a habitable or productive environment there.\n\nMoon Harvest is a game where you take the role of a Chief Operating Officer in charge of the Moon Colony Mission. \n\nThis game is powered by a complex (still in development) moon environment simulator. This complexity along with the hosts variables involved should create some awareness of how hard your quest will be.\n\nYour final goal is to create a sustainable space industry on the surface of the Moon using all resources available.\n\nYour main objetives will be:\n  - Create a sustainable process in terms of Energy.\n  - Install observatory instruments to take Astronomy to the next level.\n  - Install defensive mechanisms to protect your colony and Earth from Meteor Showers.\n  - And finally send living beings and Humans from Earth to test the environment you've created.\n\nBuild a Robot Factory, extract moon rocks, retrieve other resources such as Alumina, Silica and  He-3 to create other buildings that will help you through your journey.\n\nDuring this mission you'll get the counsel of five advisors that will guide you to achieve your goal. Each one with his own perspective of what's best. Which one will you follow?\n\n## Think, build, become the first man to successfully establish a colony on the Moon and have lots of Fun ! That's what Moon Harvest has for you!\n ","isFork":None},{"name":"buho","url":"https://github.com/cberkhoff/buho","description":None,"lang":[],"readme":"# buho","isFork":None},{"name":"mine.js","url":"https://github.com/cberkhoff/mine.js","description":"A simple game concept written in coffeescript","lang":["CoffeeScript","JavaScript"],"readme":None,"isFork":None},{"name":"winlepp","url":"https://github.com/cberkhoff/winlepp","description":"Winlepp was created as a part of a course at the Universidad de Chile. The application is focused in parallel triangle mesh refinement algorithms. It is written in C++ using QT Framework and the Eclipse IDE","lang":["Prolog","C++"],"readme":"winlepp\n=======\n\nWinlepp was created as a part of a course at the Universidad de Chile. The application is focused in parallel triangle mesh refinement algorithms. It is written in C++ using QT Framework and the Eclipse IDE","isFork":None},{"name":"android-location-picker","url":"https://github.com/cberkhoff/android-location-picker","description":"Dynamic Android control which first shows a list of countries and then, depending on the selected values, shows the corresponding child regions using dropdowns. Contains Chilean regional subdivisions. Uses Gradle.","lang":["Groovy","Shell","Java"],"readme":"android-location-picker\n=======================\n\nDynamic Android control which first shows a list of countries and then, depending on the selected values, shows the corresponding child regions using dropdowns. Contains Chilean regional subdivisions. Uses Gradle.\n","isFork":None},{"name":"facebook-android-sdk","url":"https://github.com/cberkhoff/facebook-android-sdk","description":"Facebook SDK for Android","lang":["Groovy","Java","Shell"],"readme":None,"isFork":True},{"name":"blackjack","url":"https://github.com/cberkhoff/blackjack","description":"Clojure example for an introductory worshop. It's the classic carioca card game.","lang":["Clojure"],"readme":"Blackjack\n=========\n\nClojure example for an introductory worshop. It's a simplification of the classic Blackjack card game.\n\nFor a game explanation go to [this links](http://www.pagat.com/banking/blackjack.html).\n","isFork":None},{"name":"hunchator","url":"https://github.com/imella/hunchator","description":None,"lang":["Ruby","CoffeeScript","JavaScript"],"readme":None,"isFork":None},{"name":"team11","url":"https://github.com/spaceappchile/team11","description":None,"lang":["CoffeeScript","JavaScript"],"readme":"# How I Shell NASA Project\n\n* Daniel Aguilar\n* Christian Berkhoff\n* Joaquín Jaramillo\n* Ignacio Mella\n* Sebastían Rokas\n\n## Instalación\n\nEl único requisito es _node.js_\n\n    npm -g install handlebars coffee-script\n\n## Compilación\n\n    make\n\nEste comando compila `coffe/*.coffee` a `js/app.js` y `views\\*.hb` a `js/templates.js`\n\n# Moon Harvest\n\n## Overview\n\n### Do you have what's it takes to establish a colony on the Moon? What do you think it's best for the Moon?\n\nMan have always been fascinated with our natural satellite and there are a number of reasons to create a habitable or productive environment there.\n\nMoon Harvest is a game where you take the role of a Chief Operating Officer in charge of the Moon Colony Mission. \n\nThis game is powered by a complex (still in development) moon environment simulator. This complexity along with the hosts variables involved should create some awareness of how hard your quest will be.\n\nYour final goal is to create a sustainable space industry on the surface of the Moon using all resources available.\n\nYour main objetives will be:\n  - Create a sustainable process in terms of Energy.\n  - Install observatory instruments to take Astronomy to the next level.\n  - Install defensive mechanisms to protect your colony and Earth from Meteor Showers.\n  - And finally send living beings and Humans from Earth to test the environment you've created.\n\nBuild a Robot Factory, extract moon rocks, retrieve other resources such as Alumina, Silica and  He-3 to create other buildings that will help you through your journey.\n\nDuring this mission you'll get the counsel of five advisors that will guide you to achieve your goal. Each one with his own perspective of what's best. Which one will you follow?\n\n## Think, build, become the first man to successfully establish a colony on the Moon and have lots of Fun ! That's what Moon Harvest has for you!\n ","isFork":None},{"name":"schnaps","url":"https://github.com/imella/schnaps","description":None,"lang":["CoffeeScript","JavaScript"],"readme":"# Schnaps\n\nSchaps, what could I drink tonight?","isFork":None},{"name":"euler","url":"https://github.com/cberkhoff/euler","description":"Solution to ProjectEuler problems using Java","lang":["Java"],"readme":"euler\n=====\n\nSolution to ProjectEuler problems using Java\n","isFork":None},{"name":"cc51s","url":"https://github.com/cberkhoff/cc51s","description":"A simple game written in Java using RMI for the CC51S course","lang":["Java"],"readme":"cc51s\n=====\n\nA simple game written in Java using RMI for the CC51S course","isFork":None},{"name":"UrlImageViewHelper","url":"https://github.com/cberkhoff/UrlImageViewHelper","description":"Android library that sets an ImageView's contents from a url. Manages image downloading, caching, and makes your coffee too.","lang":["Groovy","Java"],"readme":"## UrlImageViewHelper\nUrlImageViewHelper will fill an ImageView with an image that is found at a URL.\n\n### Sample Project\n\nThe sample will do a Google Image Search and load/show the results asynchronously.\n\n![](https://raw.github.com/koush/UrlImageViewHelper/master/helper2.png)\n\n### Download\n\nDownload [the latest JAR](http://repository.sonatype.org/service/local/artifact/maven/redirect?r=central-proxy&g=com.koushikdutta.urlimageviewhelper&a=urlimageviewhelper&v=LATEST\n) or grab via Maven:\n\n```xml\n<dependency>\n    <groupId>com.koushikdutta.urlimageviewhelper</groupId>\n    <artifactId>urlimageviewhelper</artifactId>\n    <version>(insert latest version)</version>\n</dependency>\n```\n\n### Usage\n\nUrlImageViewHelper will automatically download and manage all the web images and ImageViews.\nDuplicate urls will not be loaded into memory twice. Bitmap memory is managed by using\na weak reference hash table, so as soon as the image is no longer used by you,\nit will be garbage collected automatically.\n\nUsage is simple:\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\");\n```\n\n\nWant a placeholder image while it is being downloaded?\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", R.drawable.placeholder);\n```\n\n\nDon't want to use a placeholder resource, but a drawable instead?\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", drawable);\n```\n\n\nWhat if you want to preload images for snazzy fast loading?\n\n```java\nUrlImageViewHelper.loadUrlDrawable(context, \"http://example.com/image.png\");\n```\n\n\nWhat if you only want to cache the images for a minute?\n\n```java\n// Note that the 3rd argument \"null\" is an optional interstitial\n// placeholder image.\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", null, 60000);\n```\n\nUrlImageViewHelper is pretty smart. It can even load the photo for an Android contact\nif given a Contact Content Provider URI.\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"content://com.android.contacts/contacts/1115\", R.drawable.dummy_contact_photo);\n```\n\n### FAQ\n\n**Does it work in list adapters when views are reused? (convertView)**\n\nYes.\n\n\n### Featured Implementations\n\n * [ROM Manager](https://play.google.com/store/apps/details?id=com.koushikdutta.rommanager&hl=en)\n * [Carbon](https://play.google.com/store/apps/details?id=com.koushikdutta.backup&hl=en)\n * Let me know if you use this library, so I can add it to the list!\n","isFork":True},{"name":"ActionBarSherlock","url":"https://github.com/cberkhoff/ActionBarSherlock","description":"Library for implementing the action bar design pattern using the native action bar on Android 4.0+ and a custom implementation on pre-4.0 through a single API and theme.","lang":[],"readme":"ActionBarSherlock\n=================\n\nActionBarSherlock is an standalone library designed to facilitate the use of\nthe action bar design pattern across all versions of Android through a single\nAPI.\n\nThe library will automatically use the [native ActionBar][2] implementation on\nAndroid 4.0 or later. For previous versions which do not include ActionBar, a\ncustom action bar implementation based on the sources of Ice Cream Sandwich\nwill automatically be wrapped around the layout. This allows you to easily\ndevelop an application with an action bar for every version of Android from 2.x\nand up.\n\n**See http://actionbarsherlock.com for more information.**\n\n![Example Image][3]\n\nTry out the sample applications on the Android Market: [Feature Demos][4],\n[Fragments][5], and [RoboGuice][6].\n\nContinuous integration is provided by [Travis CI][7].\n\n\n\nDeveloped By\n============\n\n* Jake Wharton - <jakewharton@gmail.com>\n\n\n\nLicense\n=======\n\n    Copyright 2012 Jake Wharton\n\n    Licensed under the Apache License, Version 2.0 (the \"License\");\n    you may not use this file except in compliance with the License.\n    You may obtain a copy of the License at\n\n       http://www.apache.org/licenses/LICENSE-2.0\n\n    Unless required by applicable law or agreed to in writing, software\n    distributed under the License is distributed on an \"AS IS\" BASIS,\n    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n    See the License for the specific language governing permissions and\n    limitations under the License.\n\n\n\n\n\n [1]: http://android-developers.blogspot.com/2011/03/fragments-for-all.html\n [2]: http://developer.android.com/guide/topics/ui/actionbar.html\n [3]: http://actionbarsherlock.com/static/feature.png\n [4]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.demos\n [5]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.fragments\n [6]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.roboguice\n [7]: https://travis-ci.org/JakeWharton/ActionBarSherlock\n","isFork":True},{"name":"emacs-clojure-vagrant","url":"https://github.com/cberkhoff/emacs-clojure-vagrant","description":"A simple dev environment for clojure.","lang":["Ruby","Shell","Emacs Lisp"],"readme":"Vagrant setup for Clojure\n=========================\n\nStrated as a simple fork from [https://github.com/Seajure/emacs-clojure-vagrant](Seajure) but ended being a completedly different thing.\n\nSetup\n=====\n   - Install Vagrant following the instructions [http://vagrantup.com/docs/getting-started/index.html](Vagrant Getting Started).\n   - Download this\n   - `vagrant up` inside the directory containing this readme\n\nUsage\n-----\n\nContents\n--------\n- Java\n- Git\n- Leiningen\n- Emacs\n- Postgresql","isFork":True},{"name":"node-vagrant","url":"https://github.com/cberkhoff/node-vagrant","description":"A quick and dirty node setup using vagrant and chef","lang":["Ruby"],"readme":None,"isFork":True},{"name":"express-lingua","url":"https://github.com/cberkhoff/express-lingua","description":"A i18n middleware for the Express.js framework.","lang":["JavaScript"],"readme":"# Lingua and Lengua\n\n[Lingua](https://github.com/akoenig/express-lingua) is a middleware for the Express.js framework that helps you to internationalise your webapp easily. It determines the language of the user agent and pushes the i18n resources to your views.\n\nLengua is a fork form Lingua which introduces some features.\n\n## Installation\n\n    $ npm install lengua\n\n## Show me the code!\n\nTo translate somenthig (Coffeekup)\n\n<pre>\n@t 'Name'\n@t 'navigation.german'\n</pre>\n\nTo get the current country code, language and locale respectively\n\n<pre>\n@currentCountry\n@currentCountryLanguage\n@currentLocale\n</pre>\n\n## Added features\n\n### For the common mortal\n\n1. User locale stored in session instead of cookie\n\n2. Not case sensitive\n\n3. Country Code\n\n4. If the current locale (e.g. en-US) doesnt define a key, Lengua does a lookup in the global locale (e.g. en).\n\n### Other Stuff\n\n1. Tests :)\n\n## TODO\n\n1. Locale storage as strategy (Cookie, Session, Azure, etc).\n\n2. Cache update (when all fallbacks fail).\n\n## Dev\n\n### Tests\n\n<pre>\nnpm install jasmine-node\ncd spec/\njasmine-node.cmd .\\guru.spec.js\njasmine-node.cmd .\\lingua.spec.js\n</pre>\n\n## License\n\n[MIT License](http://www.opensource.org/licenses/mit-license.php)\n\n## Author\n\nCopyright (c) 2011,\n \n[André König](http://lochkartenstanzer.de) ([Google+](http://profile.lochkartenstanzer.de)) (andre.koenig -[at]- gmail [*dot*] com)\n\n[Christian Berkhoff](#) (christian.berkhoff - [at] - gmail [#dot#] com)\n","isFork":True},{"name":"paperclipdb","url":"https://github.com/cberkhoff/paperclipdb","description":"Database storage support for paperclip file attachment plugin.","lang":["Ruby"],"readme":None,"isFork":True}]}
,{"match":1,"linkedin_id":213954281,"gid":15986,"git_name":"Erin Krengel","git_login":"ekrengel","git_location":"Seattle, WA","git_company":None,"git_org":[],"git_email":None,"git_blog":None,"git_websiteUrl":None,"github_url":"https://github.com/ekrengel","bio":None,"repos":[{"name":"introk8s","url":"https://github.com/ekrengel/introk8s","description":"ACT-W Introduction to K8s","lang":["Dockerfile","Go"],"readme":"# ACT-W Introduction to K8s\n\nThis workshop was developed for ACT-W Seattle 2018.\n\n## Pre-Workshop Installations\n\nThere is a number of open-source software you will need to install to follow along with the workshop.\n\n### Minikube\n\nMinikube is a tool that allows you to run Kubernetes locally on your computer.\n\nYou can find install instructions for minikube [here](https://github.com/kubernetes/minikube/releases). If you use homebrew the easiest thing to do is run this command:\n\n```bash\nbrew cask install minikube\n```\n\nYou can verify you have installed minikube by running:\n\n```bash\nminikube version\n```\n\n### Kubectl\n\nKubectl is the command-line tool for interfacing with Kubernetes. It allows you to deploy and manage your apps.\n\nInstallation instructions can be found [here](https://kubernetes.io/docs/tasks/tools/install-kubectl/). Once again our recommendation is to use homebrew for this:\n\n```bash\nbrew install kubernetes-cli\n```\n\nYou can verify you have installed kubectl by running:\n\n```bash\nkubectl version\n```\n\n## Getting Started\n\n1. Start your minikube by running:\n\n    ```bash\n    minikube start\n    ```\n\n2. Make sure your context is set to minikube.\n\n    ```bash\n    kubectl config get-contexts\n    ```\n\n    `minikube` should have a star next to it when you run this command. If it doesn't, you can set minikube to your context by running:\n\n    ```bash\n    kubectl config use-context minikube\n    ```","isFork":None},{"name":"kubernetes","url":"https://github.com/ekrengel/kubernetes","description":"Production-Grade Container Scheduling and Management","lang":["Python","Makefile","Shell","C","Go","Ruby","Nginx","SaltStack","PowerShell","Protocol Buffer","HTML"],"readme":"# Kubernetes\n\n[![Submit Queue Widget]][Submit Queue] [![GoDoc Widget]][GoDoc]\n\n<img src=\"https://github.com/kubernetes/kubernetes/raw/master/logo/logo.png\" width=\"100\">\n\n----\n\nKubernetes is an open source system for managing [containerized applications]\nacross multiple hosts, providing basic mechanisms for deployment, maintenance,\nand scaling of applications.\n\nKubernetes builds upon a decade and a half of experience at Google running\nproduction workloads at scale using a system called [Borg],\ncombined with best-of-breed ideas and practices from the community.\n\nKubernetes is hosted by the Cloud Native Computing Foundation ([CNCF]).\nIf you are a company that wants to help shape the evolution of\ntechnologies that are container-packaged, dynamically-scheduled\nand microservices-oriented, consider joining the CNCF.\nFor details about who's involved and how Kubernetes plays a role,\nread the CNCF [announcement].\n\n----\n\n## To start using Kubernetes\n\nSee our documentation on [kubernetes.io].\n\nTry our [interactive tutorial].\n\nTake a free course on [Scalable Microservices with Kubernetes].\n\n## To start developing Kubernetes\n\nThe [community repository] hosts all information about\nbuilding Kubernetes from source, how to contribute code\nand documentation, who to contact about what, etc.\n\nIf you want to build Kubernetes right away there are two options:\n\n##### You have a working [Go environment].\n\n```\n$ go get -d k8s.io/kubernetes\n$ cd $GOPATH/src/k8s.io/kubernetes\n$ make\n```\n\n##### You have a working [Docker environment].\n\n```\n$ git clone https://github.com/kubernetes/kubernetes\n$ cd kubernetes\n$ make quick-release\n```\n\nIf you are less impatient, head over to the [developer's documentation].\n\n## Support\n\nIf you need support, start with the [troubleshooting guide]\nand work your way through the process that we've outlined.\n\nThat said, if you have questions, reach out to us\n[one way or another][communication].\n\n[announcement]: https://cncf.io/news/announcement/2015/07/new-cloud-native-computing-foundation-drive-alignment-among-container\n[Borg]: https://research.google.com/pubs/pub43438.html\n[CNCF]: https://www.cncf.io/about\n[communication]: https://github.com/kubernetes/community/blob/master/communication.md\n[community repository]: https://github.com/kubernetes/community\n[containerized applications]: https://kubernetes.io/docs/concepts/overview/what-is-kubernetes/\n[developer's documentation]: https://github.com/kubernetes/community/tree/master/contributors/devel\n[Docker environment]: https://docs.docker.com/engine\n[Go environment]: https://golang.org/doc/install\n[GoDoc]: https://godoc.org/k8s.io/kubernetes\n[GoDoc Widget]: https://godoc.org/k8s.io/kubernetes?status.svg\n[interactive tutorial]: http://kubernetes.io/docs/tutorials/kubernetes-basics\n[kubernetes.io]: http://kubernetes.io\n[Scalable Microservices with Kubernetes]: https://www.udacity.com/course/scalable-microservices-with-kubernetes--ud615\n[Submit Queue]: http://submit-queue.k8s.io/#/ci\n[Submit Queue Widget]: http://submit-queue.k8s.io/health.svg?v=1\n[troubleshooting guide]: https://kubernetes.io/docs/tasks/debug-application-cluster/troubleshooting/ \n\n[![Analytics](https://kubernetes-site.appspot.com/UA-36037335-10/GitHub/README.md?pixel)]()\n","isFork":True},{"name":"corvus","url":"https://github.com/pavmohan/corvus","description":"Track and share your favorite cheeses","lang":["Python","CSS","JavaScript","C","HTML"],"readme":"# Corvus\nTrack and share your favorite cheeses","isFork":None},{"name":"cs290-assignment4-part1","url":"https://github.com/ekrengel/cs290-assignment4-part1","description":"Basic PHP Assignment","lang":["PHP"],"readme":None,"isFork":None},{"name":"cs290-assignment1","url":"https://github.com/ekrengel/cs290-assignment1","description":"CS290 Assignment 1 Into to Git","lang":[],"readme":None,"isFork":None},{"name":"271-Assignments","url":"https://github.com/ekrengel/271-Assignments","description":"Assembly Language Assignments","lang":["Assembly"],"readme":None,"isFork":None},{"name":"cs290-assignment3","url":"https://github.com/ekrengel/cs290-assignment3","description":"Basic JavaScript Assignment","lang":["JavaScript","CSS","HTML"],"readme":"Basic JavaScript Assignment\n===========================\n\nThis part of the assignment will cover fundamental JavaScript topics outside the context of web development. Fully automated tests will be provided for all assignment components. These are a tool to help you assess if your code is working as intended. Passing the tests does not guarantee a good grade nor does failing them guarantee a poor grade. The grade will be based on the extent to which you meet the requirements for code in this class and the extent to which the requirements in the source files are met.\n\nWork Log\n--------\nIn order to be eligible to receive an A you must submit a work log. It will not contribute points directly to the assignment, but without it the maximum grade you will be eligible for is an 89%. The purpose is to allow me to tune the difficulty of assignments to ensure that you are getting the amount of practice needed without spending too much time on any given topic.\n\nThe format is as follows:\n\n```\nStart: Wed, 7 Jan 2015 21:42:26 -0800\nEnd: Wed, 7 Jan 2015 22:42:26 -0800\nTasks: During this period I drew a diagram of the classes I will be using to implement the message passing portion of the assignment.\n\nStart: Fri, 9 Jan 2015 18:00:00 -0800\nEnd: Fri, 9 Jan 2015 20:42:26 -0800\nTasks: During this period I debugged for 1 hour to find a type and implement the send function of the cleint.\n```\n\nYou can get most of this information from the git log if you run:\n`git log --pretty=format:\"Start: %nEnd: %ad %nTasks: %s\" --date=rfc`\nYou just need to fill in the Start: time. Use the same date format. I would suggest copying and pasting the End time and just editing the time (or day if you went through midnight in a work session).\n\nHave one entry per work session.\n\nPut this log in log.txt.\n\nInstructions\n------------\n- Create a new repository called cs290-assignment3 at the location you specified in week 1.\n- Clone this repository and copy all of the contents to that repository\n  - The directory structure should look like this:\n    - cs290-assignment3\n      - log.txt\n      - qunit_runner.html\n      - qunit\n      - src\n      - tests\n- **You are only allowed to modify the files in the src directory.**\n  - There are some additional files, those are just a hook for us to eventually support a better testing system.\n  - The comments in the files in the src directory describe the intended behavior of the functions that you need to fill in. Please ask on the discussion boards if you are confused about the intended functionality of any function.\n  - Within those files, your code should go between the `//your code here` comments.\n  - You should replace `return undefined;` with appropriate return statements.\n- Add **cs290osu** as a collaborator. This will be an account that myself and the TAs share to access your repos.\n- To test your code open the `qunit_runner.html` web page locally. Along with the red errors check for statements saying something like `Expected 4 assertions, but 2 were run`. This means that the test had to abort early due to a error it could not recover from. That means there are two additional tests that are not even getting run.\n  - Given the nature of automated tests, it is easy to see what the expected values are and simply hard code those returns. If you do this for any portion of the assignment, you will get a 0 for the whole assignment. (For example, variableModification should work for any value, not just 42, but we are only testing the value 42. If you hardcode the values `47`, `'42'` and `'42foo'` that is a violation of this rule. )\n","isFork":None},{"name":"donneler.github.io","url":"https://github.com/ekrengel/donneler.github.io","description":None,"lang":["HTML","CSS","JavaScript"],"readme":None,"isFork":None},{"name":"cs290-assignment3p2","url":"https://github.com/ekrengel/cs290-assignment3p2","description":"Assignment 3 Part 2: Adding AJAX and HTML","lang":["JavaScript","HTML"],"readme":None,"isFork":None},{"name":"cs290-assignment2","url":"https://github.com/ekrengel/cs290-assignment2","description":"HTML and CSS","lang":["HTML","CSS"],"readme":None,"isFork":None},{"name":"CS161","url":"https://github.com/ekrengel/CS161","description":"Intro to Programming in C++","lang":["C++"],"readme":"CS161\n=====\n\nIntro to Programming in C++\n","isFork":None}]}, {"match":1,"linkedin_id":21396406,"gid":17517,"git_name":"Christian Berkhoff","git_login":"cberkhoff","git_location":"Seattle, WA","git_company":"Amazon LLC","git_org":[],"git_email":"christian.berkhoff@gmail.com","git_blog":None,"git_websiteUrl":None,"github_url":"https://github.com/cberkhoff","bio":None,"repos":[{"name":"howishell","url":"https://github.com/imella/howishell","description":"How I shell NASA Project","lang":["CoffeeScript","JavaScript"],"readme":"# How I Shell NASA Project\n\n* Daniel Aguilar\n* Christian Berkhoff\n* Joaquín Jaramillo\n* Ignacio Mella\n* Sebastían Rokas\n\n## Instalación\n\nEl único requisito es _node.js_\n\n    npm -g install handlebars coffee-script\n\n## Compilación\n\n    make\n\nEste comando compila `coffe/*.coffee` a `js/app.js` y `views\\*.hb` a `js/templates.js`\n\n# Moon Harvest\n\n## Overview\n\n### Do you have what's it takes to establish a colony on the Moon? What do you think it's best for the Moon?\n\nMan have always been fascinated with our natural satellite and there are a number of reasons to create a habitable or productive environment there.\n\nMoon Harvest is a game where you take the role of a Chief Operating Officer in charge of the Moon Colony Mission. \n\nThis game is powered by a complex (still in development) moon environment simulator. This complexity along with the hosts variables involved should create some awareness of how hard your quest will be.\n\nYour final goal is to create a sustainable space industry on the surface of the Moon using all resources available.\n\nYour main objetives will be:\n  - Create a sustainable process in terms of Energy.\n  - Install observatory instruments to take Astronomy to the next level.\n  - Install defensive mechanisms to protect your colony and Earth from Meteor Showers.\n  - And finally send living beings and Humans from Earth to test the environment you've created.\n\nBuild a Robot Factory, extract moon rocks, retrieve other resources such as Alumina, Silica and  He-3 to create other buildings that will help you through your journey.\n\nDuring this mission you'll get the counsel of five advisors that will guide you to achieve your goal. Each one with his own perspective of what's best. Which one will you follow?\n\n## Think, build, become the first man to successfully establish a colony on the Moon and have lots of Fun ! That's what Moon Harvest has for you!\n ","isFork":None},{"name":"buho","url":"https://github.com/cberkhoff/buho","description":None,"lang":[],"readme":"# buho","isFork":None},{"name":"mine.js","url":"https://github.com/cberkhoff/mine.js","description":"A simple game concept written in coffeescript","lang":["CoffeeScript","JavaScript"],"readme":None,"isFork":None},{"name":"winlepp","url":"https://github.com/cberkhoff/winlepp","description":"Winlepp was created as a part of a course at the Universidad de Chile. The application is focused in parallel triangle mesh refinement algorithms. It is written in C++ using QT Framework and the Eclipse IDE","lang":["Prolog","C++"],"readme":"winlepp\n=======\n\nWinlepp was created as a part of a course at the Universidad de Chile. The application is focused in parallel triangle mesh refinement algorithms. It is written in C++ using QT Framework and the Eclipse IDE","isFork":None},{"name":"android-location-picker","url":"https://github.com/cberkhoff/android-location-picker","description":"Dynamic Android control which first shows a list of countries and then, depending on the selected values, shows the corresponding child regions using dropdowns. Contains Chilean regional subdivisions. Uses Gradle.","lang":["Groovy","Shell","Java"],"readme":"android-location-picker\n=======================\n\nDynamic Android control which first shows a list of countries and then, depending on the selected values, shows the corresponding child regions using dropdowns. Contains Chilean regional subdivisions. Uses Gradle.\n","isFork":None},{"name":"facebook-android-sdk","url":"https://github.com/cberkhoff/facebook-android-sdk","description":"Facebook SDK for Android","lang":["Groovy","Java","Shell"],"readme":None,"isFork":True},{"name":"blackjack","url":"https://github.com/cberkhoff/blackjack","description":"Clojure example for an introductory worshop. It's the classic carioca card game.","lang":["Clojure"],"readme":"Blackjack\n=========\n\nClojure example for an introductory worshop. It's a simplification of the classic Blackjack card game.\n\nFor a game explanation go to [this links](http://www.pagat.com/banking/blackjack.html).\n","isFork":None},{"name":"hunchator","url":"https://github.com/imella/hunchator","description":None,"lang":["Ruby","CoffeeScript","JavaScript"],"readme":None,"isFork":None},{"name":"team11","url":"https://github.com/spaceappchile/team11","description":None,"lang":["CoffeeScript","JavaScript"],"readme":"# How I Shell NASA Project\n\n* Daniel Aguilar\n* Christian Berkhoff\n* Joaquín Jaramillo\n* Ignacio Mella\n* Sebastían Rokas\n\n## Instalación\n\nEl único requisito es _node.js_\n\n    npm -g install handlebars coffee-script\n\n## Compilación\n\n    make\n\nEste comando compila `coffe/*.coffee` a `js/app.js` y `views\\*.hb` a `js/templates.js`\n\n# Moon Harvest\n\n## Overview\n\n### Do you have what's it takes to establish a colony on the Moon? What do you think it's best for the Moon?\n\nMan have always been fascinated with our natural satellite and there are a number of reasons to create a habitable or productive environment there.\n\nMoon Harvest is a game where you take the role of a Chief Operating Officer in charge of the Moon Colony Mission. \n\nThis game is powered by a complex (still in development) moon environment simulator. This complexity along with the hosts variables involved should create some awareness of how hard your quest will be.\n\nYour final goal is to create a sustainable space industry on the surface of the Moon using all resources available.\n\nYour main objetives will be:\n  - Create a sustainable process in terms of Energy.\n  - Install observatory instruments to take Astronomy to the next level.\n  - Install defensive mechanisms to protect your colony and Earth from Meteor Showers.\n  - And finally send living beings and Humans from Earth to test the environment you've created.\n\nBuild a Robot Factory, extract moon rocks, retrieve other resources such as Alumina, Silica and  He-3 to create other buildings that will help you through your journey.\n\nDuring this mission you'll get the counsel of five advisors that will guide you to achieve your goal. Each one with his own perspective of what's best. Which one will you follow?\n\n## Think, build, become the first man to successfully establish a colony on the Moon and have lots of Fun ! That's what Moon Harvest has for you!\n ","isFork":None},{"name":"schnaps","url":"https://github.com/imella/schnaps","description":None,"lang":["CoffeeScript","JavaScript"],"readme":"# Schnaps\n\nSchaps, what could I drink tonight?","isFork":None},{"name":"euler","url":"https://github.com/cberkhoff/euler","description":"Solution to ProjectEuler problems using Java","lang":["Java"],"readme":"euler\n=====\n\nSolution to ProjectEuler problems using Java\n","isFork":None},{"name":"cc51s","url":"https://github.com/cberkhoff/cc51s","description":"A simple game written in Java using RMI for the CC51S course","lang":["Java"],"readme":"cc51s\n=====\n\nA simple game written in Java using RMI for the CC51S course","isFork":None},{"name":"UrlImageViewHelper","url":"https://github.com/cberkhoff/UrlImageViewHelper","description":"Android library that sets an ImageView's contents from a url. Manages image downloading, caching, and makes your coffee too.","lang":["Groovy","Java"],"readme":"## UrlImageViewHelper\nUrlImageViewHelper will fill an ImageView with an image that is found at a URL.\n\n### Sample Project\n\nThe sample will do a Google Image Search and load/show the results asynchronously.\n\n![](https://raw.github.com/koush/UrlImageViewHelper/master/helper2.png)\n\n### Download\n\nDownload [the latest JAR](http://repository.sonatype.org/service/local/artifact/maven/redirect?r=central-proxy&g=com.koushikdutta.urlimageviewhelper&a=urlimageviewhelper&v=LATEST\n) or grab via Maven:\n\n```xml\n<dependency>\n    <groupId>com.koushikdutta.urlimageviewhelper</groupId>\n    <artifactId>urlimageviewhelper</artifactId>\n    <version>(insert latest version)</version>\n</dependency>\n```\n\n### Usage\n\nUrlImageViewHelper will automatically download and manage all the web images and ImageViews.\nDuplicate urls will not be loaded into memory twice. Bitmap memory is managed by using\na weak reference hash table, so as soon as the image is no longer used by you,\nit will be garbage collected automatically.\n\nUsage is simple:\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\");\n```\n\n\nWant a placeholder image while it is being downloaded?\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", R.drawable.placeholder);\n```\n\n\nDon't want to use a placeholder resource, but a drawable instead?\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", drawable);\n```\n\n\nWhat if you want to preload images for snazzy fast loading?\n\n```java\nUrlImageViewHelper.loadUrlDrawable(context, \"http://example.com/image.png\");\n```\n\n\nWhat if you only want to cache the images for a minute?\n\n```java\n// Note that the 3rd argument \"null\" is an optional interstitial\n// placeholder image.\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", null, 60000);\n```\n\nUrlImageViewHelper is pretty smart. It can even load the photo for an Android contact\nif given a Contact Content Provider URI.\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"content://com.android.contacts/contacts/1115\", R.drawable.dummy_contact_photo);\n```\n\n### FAQ\n\n**Does it work in list adapters when views are reused? (convertView)**\n\nYes.\n\n\n### Featured Implementations\n\n * [ROM Manager](https://play.google.com/store/apps/details?id=com.koushikdutta.rommanager&hl=en)\n * [Carbon](https://play.google.com/store/apps/details?id=com.koushikdutta.backup&hl=en)\n * Let me know if you use this library, so I can add it to the list!\n","isFork":True},{"name":"ActionBarSherlock","url":"https://github.com/cberkhoff/ActionBarSherlock","description":"Library for implementing the action bar design pattern using the native action bar on Android 4.0+ and a custom implementation on pre-4.0 through a single API and theme.","lang":[],"readme":"ActionBarSherlock\n=================\n\nActionBarSherlock is an standalone library designed to facilitate the use of\nthe action bar design pattern across all versions of Android through a single\nAPI.\n\nThe library will automatically use the [native ActionBar][2] implementation on\nAndroid 4.0 or later. For previous versions which do not include ActionBar, a\ncustom action bar implementation based on the sources of Ice Cream Sandwich\nwill automatically be wrapped around the layout. This allows you to easily\ndevelop an application with an action bar for every version of Android from 2.x\nand up.\n\n**See http://actionbarsherlock.com for more information.**\n\n![Example Image][3]\n\nTry out the sample applications on the Android Market: [Feature Demos][4],\n[Fragments][5], and [RoboGuice][6].\n\nContinuous integration is provided by [Travis CI][7].\n\n\n\nDeveloped By\n============\n\n* Jake Wharton - <jakewharton@gmail.com>\n\n\n\nLicense\n=======\n\n    Copyright 2012 Jake Wharton\n\n    Licensed under the Apache License, Version 2.0 (the \"License\");\n    you may not use this file except in compliance with the License.\n    You may obtain a copy of the License at\n\n       http://www.apache.org/licenses/LICENSE-2.0\n\n    Unless required by applicable law or agreed to in writing, software\n    distributed under the License is distributed on an \"AS IS\" BASIS,\n    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n    See the License for the specific language governing permissions and\n    limitations under the License.\n\n\n\n\n\n [1]: http://android-developers.blogspot.com/2011/03/fragments-for-all.html\n [2]: http://developer.android.com/guide/topics/ui/actionbar.html\n [3]: http://actionbarsherlock.com/static/feature.png\n [4]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.demos\n [5]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.fragments\n [6]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.roboguice\n [7]: https://travis-ci.org/JakeWharton/ActionBarSherlock\n","isFork":True},{"name":"emacs-clojure-vagrant","url":"https://github.com/cberkhoff/emacs-clojure-vagrant","description":"A simple dev environment for clojure.","lang":["Ruby","Shell","Emacs Lisp"],"readme":"Vagrant setup for Clojure\n=========================\n\nStrated as a simple fork from [https://github.com/Seajure/emacs-clojure-vagrant](Seajure) but ended being a completedly different thing.\n\nSetup\n=====\n   - Install Vagrant following the instructions [http://vagrantup.com/docs/getting-started/index.html](Vagrant Getting Started).\n   - Download this\n   - `vagrant up` inside the directory containing this readme\n\nUsage\n-----\n\nContents\n--------\n- Java\n- Git\n- Leiningen\n- Emacs\n- Postgresql","isFork":True},{"name":"node-vagrant","url":"https://github.com/cberkhoff/node-vagrant","description":"A quick and dirty node setup using vagrant and chef","lang":["Ruby"],"readme":None,"isFork":True},{"name":"express-lingua","url":"https://github.com/cberkhoff/express-lingua","description":"A i18n middleware for the Express.js framework.","lang":["JavaScript"],"readme":"# Lingua and Lengua\n\n[Lingua](https://github.com/akoenig/express-lingua) is a middleware for the Express.js framework that helps you to internationalise your webapp easily. It determines the language of the user agent and pushes the i18n resources to your views.\n\nLengua is a fork form Lingua which introduces some features.\n\n## Installation\n\n    $ npm install lengua\n\n## Show me the code!\n\nTo translate somenthig (Coffeekup)\n\n<pre>\n@t 'Name'\n@t 'navigation.german'\n</pre>\n\nTo get the current country code, language and locale respectively\n\n<pre>\n@currentCountry\n@currentCountryLanguage\n@currentLocale\n</pre>\n\n## Added features\n\n### For the common mortal\n\n1. User locale stored in session instead of cookie\n\n2. Not case sensitive\n\n3. Country Code\n\n4. If the current locale (e.g. en-US) doesnt define a key, Lengua does a lookup in the global locale (e.g. en).\n\n### Other Stuff\n\n1. Tests :)\n\n## TODO\n\n1. Locale storage as strategy (Cookie, Session, Azure, etc).\n\n2. Cache update (when all fallbacks fail).\n\n## Dev\n\n### Tests\n\n<pre>\nnpm install jasmine-node\ncd spec/\njasmine-node.cmd .\\guru.spec.js\njasmine-node.cmd .\\lingua.spec.js\n</pre>\n\n## License\n\n[MIT License](http://www.opensource.org/licenses/mit-license.php)\n\n## Author\n\nCopyright (c) 2011,\n \n[André König](http://lochkartenstanzer.de) ([Google+](http://profile.lochkartenstanzer.de)) (andre.koenig -[at]- gmail [*dot*] com)\n\n[Christian Berkhoff](#) (christian.berkhoff - [at] - gmail [#dot#] com)\n","isFork":True},{"name":"paperclipdb","url":"https://github.com/cberkhoff/paperclipdb","description":"Database storage support for paperclip file attachment plugin.","lang":["Ruby"],"readme":None,"isFork":True}]}
,{"match":1,"linkedin_id":213954281,"gid":15986,"git_name":"Erin Krengel","git_login":"ekrengel","git_location":"Seattle, WA","git_company":None,"git_org":[],"git_email":None,"git_blog":None,"git_websiteUrl":None,"github_url":"https://github.com/ekrengel","bio":None,"repos":[{"name":"introk8s","url":"https://github.com/ekrengel/introk8s","description":"ACT-W Introduction to K8s","lang":["Dockerfile","Go"],"readme":"# ACT-W Introduction to K8s\n\nThis workshop was developed for ACT-W Seattle 2018.\n\n## Pre-Workshop Installations\n\nThere is a number of open-source software you will need to install to follow along with the workshop.\n\n### Minikube\n\nMinikube is a tool that allows you to run Kubernetes locally on your computer.\n\nYou can find install instructions for minikube [here](https://github.com/kubernetes/minikube/releases). If you use homebrew the easiest thing to do is run this command:\n\n```bash\nbrew cask install minikube\n```\n\nYou can verify you have installed minikube by running:\n\n```bash\nminikube version\n```\n\n### Kubectl\n\nKubectl is the command-line tool for interfacing with Kubernetes. It allows you to deploy and manage your apps.\n\nInstallation instructions can be found [here](https://kubernetes.io/docs/tasks/tools/install-kubectl/). Once again our recommendation is to use homebrew for this:\n\n```bash\nbrew install kubernetes-cli\n```\n\nYou can verify you have installed kubectl by running:\n\n```bash\nkubectl version\n```\n\n## Getting Started\n\n1. Start your minikube by running:\n\n    ```bash\n    minikube start\n    ```\n\n2. Make sure your context is set to minikube.\n\n    ```bash\n    kubectl config get-contexts\n    ```\n\n    `minikube` should have a star next to it when you run this command. If it doesn't, you can set minikube to your context by running:\n\n    ```bash\n    kubectl config use-context minikube\n    ```","isFork":None},{"name":"kubernetes","url":"https://github.com/ekrengel/kubernetes","description":"Production-Grade Container Scheduling and Management","lang":["Python","Makefile","Shell","C","Go","Ruby","Nginx","SaltStack","PowerShell","Protocol Buffer","HTML"],"readme":"# Kubernetes\n\n[![Submit Queue Widget]][Submit Queue] [![GoDoc Widget]][GoDoc]\n\n<img src=\"https://github.com/kubernetes/kubernetes/raw/master/logo/logo.png\" width=\"100\">\n\n----\n\nKubernetes is an open source system for managing [containerized applications]\nacross multiple hosts, providing basic mechanisms for deployment, maintenance,\nand scaling of applications.\n\nKubernetes builds upon a decade and a half of experience at Google running\nproduction workloads at scale using a system called [Borg],\ncombined with best-of-breed ideas and practices from the community.\n\nKubernetes is hosted by the Cloud Native Computing Foundation ([CNCF]).\nIf you are a company that wants to help shape the evolution of\ntechnologies that are container-packaged, dynamically-scheduled\nand microservices-oriented, consider joining the CNCF.\nFor details about who's involved and how Kubernetes plays a role,\nread the CNCF [announcement].\n\n----\n\n## To start using Kubernetes\n\nSee our documentation on [kubernetes.io].\n\nTry our [interactive tutorial].\n\nTake a free course on [Scalable Microservices with Kubernetes].\n\n## To start developing Kubernetes\n\nThe [community repository] hosts all information about\nbuilding Kubernetes from source, how to contribute code\nand documentation, who to contact about what, etc.\n\nIf you want to build Kubernetes right away there are two options:\n\n##### You have a working [Go environment].\n\n```\n$ go get -d k8s.io/kubernetes\n$ cd $GOPATH/src/k8s.io/kubernetes\n$ make\n```\n\n##### You have a working [Docker environment].\n\n```\n$ git clone https://github.com/kubernetes/kubernetes\n$ cd kubernetes\n$ make quick-release\n```\n\nIf you are less impatient, head over to the [developer's documentation].\n\n## Support\n\nIf you need support, start with the [troubleshooting guide]\nand work your way through the process that we've outlined.\n\nThat said, if you have questions, reach out to us\n[one way or another][communication].\n\n[announcement]: https://cncf.io/news/announcement/2015/07/new-cloud-native-computing-foundation-drive-alignment-among-container\n[Borg]: https://research.google.com/pubs/pub43438.html\n[CNCF]: https://www.cncf.io/about\n[communication]: https://github.com/kubernetes/community/blob/master/communication.md\n[community repository]: https://github.com/kubernetes/community\n[containerized applications]: https://kubernetes.io/docs/concepts/overview/what-is-kubernetes/\n[developer's documentation]: https://github.com/kubernetes/community/tree/master/contributors/devel\n[Docker environment]: https://docs.docker.com/engine\n[Go environment]: https://golang.org/doc/install\n[GoDoc]: https://godoc.org/k8s.io/kubernetes\n[GoDoc Widget]: https://godoc.org/k8s.io/kubernetes?status.svg\n[interactive tutorial]: http://kubernetes.io/docs/tutorials/kubernetes-basics\n[kubernetes.io]: http://kubernetes.io\n[Scalable Microservices with Kubernetes]: https://www.udacity.com/course/scalable-microservices-with-kubernetes--ud615\n[Submit Queue]: http://submit-queue.k8s.io/#/ci\n[Submit Queue Widget]: http://submit-queue.k8s.io/health.svg?v=1\n[troubleshooting guide]: https://kubernetes.io/docs/tasks/debug-application-cluster/troubleshooting/ \n\n[![Analytics](https://kubernetes-site.appspot.com/UA-36037335-10/GitHub/README.md?pixel)]()\n","isFork":True},{"name":"corvus","url":"https://github.com/pavmohan/corvus","description":"Track and share your favorite cheeses","lang":["Python","CSS","JavaScript","C","HTML"],"readme":"# Corvus\nTrack and share your favorite cheeses","isFork":None},{"name":"cs290-assignment4-part1","url":"https://github.com/ekrengel/cs290-assignment4-part1","description":"Basic PHP Assignment","lang":["PHP"],"readme":None,"isFork":None},{"name":"cs290-assignment1","url":"https://github.com/ekrengel/cs290-assignment1","description":"CS290 Assignment 1 Into to Git","lang":[],"readme":None,"isFork":None},{"name":"271-Assignments","url":"https://github.com/ekrengel/271-Assignments","description":"Assembly Language Assignments","lang":["Assembly"],"readme":None,"isFork":None},{"name":"cs290-assignment3","url":"https://github.com/ekrengel/cs290-assignment3","description":"Basic JavaScript Assignment","lang":["JavaScript","CSS","HTML"],"readme":"Basic JavaScript Assignment\n===========================\n\nThis part of the assignment will cover fundamental JavaScript topics outside the context of web development. Fully automated tests will be provided for all assignment components. These are a tool to help you assess if your code is working as intended. Passing the tests does not guarantee a good grade nor does failing them guarantee a poor grade. The grade will be based on the extent to which you meet the requirements for code in this class and the extent to which the requirements in the source files are met.\n\nWork Log\n--------\nIn order to be eligible to receive an A you must submit a work log. It will not contribute points directly to the assignment, but without it the maximum grade you will be eligible for is an 89%. The purpose is to allow me to tune the difficulty of assignments to ensure that you are getting the amount of practice needed without spending too much time on any given topic.\n\nThe format is as follows:\n\n```\nStart: Wed, 7 Jan 2015 21:42:26 -0800\nEnd: Wed, 7 Jan 2015 22:42:26 -0800\nTasks: During this period I drew a diagram of the classes I will be using to implement the message passing portion of the assignment.\n\nStart: Fri, 9 Jan 2015 18:00:00 -0800\nEnd: Fri, 9 Jan 2015 20:42:26 -0800\nTasks: During this period I debugged for 1 hour to find a type and implement the send function of the cleint.\n```\n\nYou can get most of this information from the git log if you run:\n`git log --pretty=format:\"Start: %nEnd: %ad %nTasks: %s\" --date=rfc`\nYou just need to fill in the Start: time. Use the same date format. I would suggest copying and pasting the End time and just editing the time (or day if you went through midnight in a work session).\n\nHave one entry per work session.\n\nPut this log in log.txt.\n\nInstructions\n------------\n- Create a new repository called cs290-assignment3 at the location you specified in week 1.\n- Clone this repository and copy all of the contents to that repository\n  - The directory structure should look like this:\n    - cs290-assignment3\n      - log.txt\n      - qunit_runner.html\n      - qunit\n      - src\n      - tests\n- **You are only allowed to modify the files in the src directory.**\n  - There are some additional files, those are just a hook for us to eventually support a better testing system.\n  - The comments in the files in the src directory describe the intended behavior of the functions that you need to fill in. Please ask on the discussion boards if you are confused about the intended functionality of any function.\n  - Within those files, your code should go between the `//your code here` comments.\n  - You should replace `return undefined;` with appropriate return statements.\n- Add **cs290osu** as a collaborator. This will be an account that myself and the TAs share to access your repos.\n- To test your code open the `qunit_runner.html` web page locally. Along with the red errors check for statements saying something like `Expected 4 assertions, but 2 were run`. This means that the test had to abort early due to a error it could not recover from. That means there are two additional tests that are not even getting run.\n  - Given the nature of automated tests, it is easy to see what the expected values are and simply hard code those returns. If you do this for any portion of the assignment, you will get a 0 for the whole assignment. (For example, variableModification should work for any value, not just 42, but we are only testing the value 42. If you hardcode the values `47`, `'42'` and `'42foo'` that is a violation of this rule. )\n","isFork":None},{"name":"donneler.github.io","url":"https://github.com/ekrengel/donneler.github.io","description":None,"lang":["HTML","CSS","JavaScript"],"readme":None,"isFork":None},{"name":"cs290-assignment3p2","url":"https://github.com/ekrengel/cs290-assignment3p2","description":"Assignment 3 Part 2: Adding AJAX and HTML","lang":["JavaScript","HTML"],"readme":None,"isFork":None},{"name":"cs290-assignment2","url":"https://github.com/ekrengel/cs290-assignment2","description":"HTML and CSS","lang":["HTML","CSS"],"readme":None,"isFork":None},{"name":"CS161","url":"https://github.com/ekrengel/CS161","description":"Intro to Programming in C++","lang":["C++"],"readme":"CS161\n=====\n\nIntro to Programming in C++\n","isFork":None}]}, {"match":1,"linkedin_id":213964096,"gid":17517,"git_name":"Christian Berkhoff","git_login":"cberkhoff","git_location":"Seattle, WA","git_company":"Amazon LLC","git_org":[],"git_email":"christian.berkhoff@gmail.com","git_blog":None,"git_websiteUrl":None,"github_url":"https://github.com/cberkhoff","bio":None,"repos":[{"name":"howishell","url":"https://github.com/imella/howishell","description":"How I shell NASA Project","lang":["CoffeeScript","JavaScript"],"readme":"# How I Shell NASA Project\n\n* Daniel Aguilar\n* Christian Berkhoff\n* Joaquín Jaramillo\n* Ignacio Mella\n* Sebastían Rokas\n\n## Instalación\n\nEl único requisito es _node.js_\n\n    npm -g install handlebars coffee-script\n\n## Compilación\n\n    make\n\nEste comando compila `coffe/*.coffee` a `js/app.js` y `views\\*.hb` a `js/templates.js`\n\n# Moon Harvest\n\n## Overview\n\n### Do you have what's it takes to establish a colony on the Moon? What do you think it's best for the Moon?\n\nMan have always been fascinated with our natural satellite and there are a number of reasons to create a habitable or productive environment there.\n\nMoon Harvest is a game where you take the role of a Chief Operating Officer in charge of the Moon Colony Mission. \n\nThis game is powered by a complex (still in development) moon environment simulator. This complexity along with the hosts variables involved should create some awareness of how hard your quest will be.\n\nYour final goal is to create a sustainable space industry on the surface of the Moon using all resources available.\n\nYour main objetives will be:\n  - Create a sustainable process in terms of Energy.\n  - Install observatory instruments to take Astronomy to the next level.\n  - Install defensive mechanisms to protect your colony and Earth from Meteor Showers.\n  - And finally send living beings and Humans from Earth to test the environment you've created.\n\nBuild a Robot Factory, extract moon rocks, retrieve other resources such as Alumina, Silica and  He-3 to create other buildings that will help you through your journey.\n\nDuring this mission you'll get the counsel of five advisors that will guide you to achieve your goal. Each one with his own perspective of what's best. Which one will you follow?\n\n## Think, build, become the first man to successfully establish a colony on the Moon and have lots of Fun ! That's what Moon Harvest has for you!\n ","isFork":None},{"name":"buho","url":"https://github.com/cberkhoff/buho","description":None,"lang":[],"readme":"# buho","isFork":None},{"name":"mine.js","url":"https://github.com/cberkhoff/mine.js","description":"A simple game concept written in coffeescript","lang":["CoffeeScript","JavaScript"],"readme":None,"isFork":None},{"name":"winlepp","url":"https://github.com/cberkhoff/winlepp","description":"Winlepp was created as a part of a course at the Universidad de Chile. The application is focused in parallel triangle mesh refinement algorithms. It is written in C++ using QT Framework and the Eclipse IDE","lang":["Prolog","C++"],"readme":"winlepp\n=======\n\nWinlepp was created as a part of a course at the Universidad de Chile. The application is focused in parallel triangle mesh refinement algorithms. It is written in C++ using QT Framework and the Eclipse IDE","isFork":None},{"name":"android-location-picker","url":"https://github.com/cberkhoff/android-location-picker","description":"Dynamic Android control which first shows a list of countries and then, depending on the selected values, shows the corresponding child regions using dropdowns. Contains Chilean regional subdivisions. Uses Gradle.","lang":["Groovy","Shell","Java"],"readme":"android-location-picker\n=======================\n\nDynamic Android control which first shows a list of countries and then, depending on the selected values, shows the corresponding child regions using dropdowns. Contains Chilean regional subdivisions. Uses Gradle.\n","isFork":None},{"name":"facebook-android-sdk","url":"https://github.com/cberkhoff/facebook-android-sdk","description":"Facebook SDK for Android","lang":["Groovy","Java","Shell"],"readme":None,"isFork":True},{"name":"blackjack","url":"https://github.com/cberkhoff/blackjack","description":"Clojure example for an introductory worshop. It's the classic carioca card game.","lang":["Clojure"],"readme":"Blackjack\n=========\n\nClojure example for an introductory worshop. It's a simplification of the classic Blackjack card game.\n\nFor a game explanation go to [this links](http://www.pagat.com/banking/blackjack.html).\n","isFork":None},{"name":"hunchator","url":"https://github.com/imella/hunchator","description":None,"lang":["Ruby","CoffeeScript","JavaScript"],"readme":None,"isFork":None},{"name":"team11","url":"https://github.com/spaceappchile/team11","description":None,"lang":["CoffeeScript","JavaScript"],"readme":"# How I Shell NASA Project\n\n* Daniel Aguilar\n* Christian Berkhoff\n* Joaquín Jaramillo\n* Ignacio Mella\n* Sebastían Rokas\n\n## Instalación\n\nEl único requisito es _node.js_\n\n    npm -g install handlebars coffee-script\n\n## Compilación\n\n    make\n\nEste comando compila `coffe/*.coffee` a `js/app.js` y `views\\*.hb` a `js/templates.js`\n\n# Moon Harvest\n\n## Overview\n\n### Do you have what's it takes to establish a colony on the Moon? What do you think it's best for the Moon?\n\nMan have always been fascinated with our natural satellite and there are a number of reasons to create a habitable or productive environment there.\n\nMoon Harvest is a game where you take the role of a Chief Operating Officer in charge of the Moon Colony Mission. \n\nThis game is powered by a complex (still in development) moon environment simulator. This complexity along with the hosts variables involved should create some awareness of how hard your quest will be.\n\nYour final goal is to create a sustainable space industry on the surface of the Moon using all resources available.\n\nYour main objetives will be:\n  - Create a sustainable process in terms of Energy.\n  - Install observatory instruments to take Astronomy to the next level.\n  - Install defensive mechanisms to protect your colony and Earth from Meteor Showers.\n  - And finally send living beings and Humans from Earth to test the environment you've created.\n\nBuild a Robot Factory, extract moon rocks, retrieve other resources such as Alumina, Silica and  He-3 to create other buildings that will help you through your journey.\n\nDuring this mission you'll get the counsel of five advisors that will guide you to achieve your goal. Each one with his own perspective of what's best. Which one will you follow?\n\n## Think, build, become the first man to successfully establish a colony on the Moon and have lots of Fun ! That's what Moon Harvest has for you!\n ","isFork":None},{"name":"schnaps","url":"https://github.com/imella/schnaps","description":None,"lang":["CoffeeScript","JavaScript"],"readme":"# Schnaps\n\nSchaps, what could I drink tonight?","isFork":None},{"name":"euler","url":"https://github.com/cberkhoff/euler","description":"Solution to ProjectEuler problems using Java","lang":["Java"],"readme":"euler\n=====\n\nSolution to ProjectEuler problems using Java\n","isFork":None},{"name":"cc51s","url":"https://github.com/cberkhoff/cc51s","description":"A simple game written in Java using RMI for the CC51S course","lang":["Java"],"readme":"cc51s\n=====\n\nA simple game written in Java using RMI for the CC51S course","isFork":None},{"name":"UrlImageViewHelper","url":"https://github.com/cberkhoff/UrlImageViewHelper","description":"Android library that sets an ImageView's contents from a url. Manages image downloading, caching, and makes your coffee too.","lang":["Groovy","Java"],"readme":"## UrlImageViewHelper\nUrlImageViewHelper will fill an ImageView with an image that is found at a URL.\n\n### Sample Project\n\nThe sample will do a Google Image Search and load/show the results asynchronously.\n\n![](https://raw.github.com/koush/UrlImageViewHelper/master/helper2.png)\n\n### Download\n\nDownload [the latest JAR](http://repository.sonatype.org/service/local/artifact/maven/redirect?r=central-proxy&g=com.koushikdutta.urlimageviewhelper&a=urlimageviewhelper&v=LATEST\n) or grab via Maven:\n\n```xml\n<dependency>\n    <groupId>com.koushikdutta.urlimageviewhelper</groupId>\n    <artifactId>urlimageviewhelper</artifactId>\n    <version>(insert latest version)</version>\n</dependency>\n```\n\n### Usage\n\nUrlImageViewHelper will automatically download and manage all the web images and ImageViews.\nDuplicate urls will not be loaded into memory twice. Bitmap memory is managed by using\na weak reference hash table, so as soon as the image is no longer used by you,\nit will be garbage collected automatically.\n\nUsage is simple:\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\");\n```\n\n\nWant a placeholder image while it is being downloaded?\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", R.drawable.placeholder);\n```\n\n\nDon't want to use a placeholder resource, but a drawable instead?\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", drawable);\n```\n\n\nWhat if you want to preload images for snazzy fast loading?\n\n```java\nUrlImageViewHelper.loadUrlDrawable(context, \"http://example.com/image.png\");\n```\n\n\nWhat if you only want to cache the images for a minute?\n\n```java\n// Note that the 3rd argument \"null\" is an optional interstitial\n// placeholder image.\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", null, 60000);\n```\n\nUrlImageViewHelper is pretty smart. It can even load the photo for an Android contact\nif given a Contact Content Provider URI.\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"content://com.android.contacts/contacts/1115\", R.drawable.dummy_contact_photo);\n```\n\n### FAQ\n\n**Does it work in list adapters when views are reused? (convertView)**\n\nYes.\n\n\n### Featured Implementations\n\n * [ROM Manager](https://play.google.com/store/apps/details?id=com.koushikdutta.rommanager&hl=en)\n * [Carbon](https://play.google.com/store/apps/details?id=com.koushikdutta.backup&hl=en)\n * Let me know if you use this library, so I can add it to the list!\n","isFork":True},{"name":"ActionBarSherlock","url":"https://github.com/cberkhoff/ActionBarSherlock","description":"Library for implementing the action bar design pattern using the native action bar on Android 4.0+ and a custom implementation on pre-4.0 through a single API and theme.","lang":[],"readme":"ActionBarSherlock\n=================\n\nActionBarSherlock is an standalone library designed to facilitate the use of\nthe action bar design pattern across all versions of Android through a single\nAPI.\n\nThe library will automatically use the [native ActionBar][2] implementation on\nAndroid 4.0 or later. For previous versions which do not include ActionBar, a\ncustom action bar implementation based on the sources of Ice Cream Sandwich\nwill automatically be wrapped around the layout. This allows you to easily\ndevelop an application with an action bar for every version of Android from 2.x\nand up.\n\n**See http://actionbarsherlock.com for more information.**\n\n![Example Image][3]\n\nTry out the sample applications on the Android Market: [Feature Demos][4],\n[Fragments][5], and [RoboGuice][6].\n\nContinuous integration is provided by [Travis CI][7].\n\n\n\nDeveloped By\n============\n\n* Jake Wharton - <jakewharton@gmail.com>\n\n\n\nLicense\n=======\n\n    Copyright 2012 Jake Wharton\n\n    Licensed under the Apache License, Version 2.0 (the \"License\");\n    you may not use this file except in compliance with the License.\n    You may obtain a copy of the License at\n\n       http://www.apache.org/licenses/LICENSE-2.0\n\n    Unless required by applicable law or agreed to in writing, software\n    distributed under the License is distributed on an \"AS IS\" BASIS,\n    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n    See the License for the specific language governing permissions and\n    limitations under the License.\n\n\n\n\n\n [1]: http://android-developers.blogspot.com/2011/03/fragments-for-all.html\n [2]: http://developer.android.com/guide/topics/ui/actionbar.html\n [3]: http://actionbarsherlock.com/static/feature.png\n [4]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.demos\n [5]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.fragments\n [6]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.roboguice\n [7]: https://travis-ci.org/JakeWharton/ActionBarSherlock\n","isFork":True},{"name":"emacs-clojure-vagrant","url":"https://github.com/cberkhoff/emacs-clojure-vagrant","description":"A simple dev environment for clojure.","lang":["Ruby","Shell","Emacs Lisp"],"readme":"Vagrant setup for Clojure\n=========================\n\nStrated as a simple fork from [https://github.com/Seajure/emacs-clojure-vagrant](Seajure) but ended being a completedly different thing.\n\nSetup\n=====\n   - Install Vagrant following the instructions [http://vagrantup.com/docs/getting-started/index.html](Vagrant Getting Started).\n   - Download this\n   - `vagrant up` inside the directory containing this readme\n\nUsage\n-----\n\nContents\n--------\n- Java\n- Git\n- Leiningen\n- Emacs\n- Postgresql","isFork":True},{"name":"node-vagrant","url":"https://github.com/cberkhoff/node-vagrant","description":"A quick and dirty node setup using vagrant and chef","lang":["Ruby"],"readme":None,"isFork":True},{"name":"express-lingua","url":"https://github.com/cberkhoff/express-lingua","description":"A i18n middleware for the Express.js framework.","lang":["JavaScript"],"readme":"# Lingua and Lengua\n\n[Lingua](https://github.com/akoenig/express-lingua) is a middleware for the Express.js framework that helps you to internationalise your webapp easily. It determines the language of the user agent and pushes the i18n resources to your views.\n\nLengua is a fork form Lingua which introduces some features.\n\n## Installation\n\n    $ npm install lengua\n\n## Show me the code!\n\nTo translate somenthig (Coffeekup)\n\n<pre>\n@t 'Name'\n@t 'navigation.german'\n</pre>\n\nTo get the current country code, language and locale respectively\n\n<pre>\n@currentCountry\n@currentCountryLanguage\n@currentLocale\n</pre>\n\n## Added features\n\n### For the common mortal\n\n1. User locale stored in session instead of cookie\n\n2. Not case sensitive\n\n3. Country Code\n\n4. If the current locale (e.g. en-US) doesnt define a key, Lengua does a lookup in the global locale (e.g. en).\n\n### Other Stuff\n\n1. Tests :)\n\n## TODO\n\n1. Locale storage as strategy (Cookie, Session, Azure, etc).\n\n2. Cache update (when all fallbacks fail).\n\n## Dev\n\n### Tests\n\n<pre>\nnpm install jasmine-node\ncd spec/\njasmine-node.cmd .\\guru.spec.js\njasmine-node.cmd .\\lingua.spec.js\n</pre>\n\n## License\n\n[MIT License](http://www.opensource.org/licenses/mit-license.php)\n\n## Author\n\nCopyright (c) 2011,\n \n[André König](http://lochkartenstanzer.de) ([Google+](http://profile.lochkartenstanzer.de)) (andre.koenig -[at]- gmail [*dot*] com)\n\n[Christian Berkhoff](#) (christian.berkhoff - [at] - gmail [#dot#] com)\n","isFork":True},{"name":"paperclipdb","url":"https://github.com/cberkhoff/paperclipdb","description":"Database storage support for paperclip file attachment plugin.","lang":["Ruby"],"readme":None,"isFork":True}]}
,{"match":1,"linkedin_id":213954281,"gid":15986,"git_name":"Erin Krengel","git_login":"ekrengel","git_location":"Seattle, WA","git_company":None,"git_org":[],"git_email":None,"git_blog":None,"git_websiteUrl":None,"github_url":"https://github.com/ekrengel","bio":None,"repos":[{"name":"introk8s","url":"https://github.com/ekrengel/introk8s","description":"ACT-W Introduction to K8s","lang":["Dockerfile","Go"],"readme":"# ACT-W Introduction to K8s\n\nThis workshop was developed for ACT-W Seattle 2018.\n\n## Pre-Workshop Installations\n\nThere is a number of open-source software you will need to install to follow along with the workshop.\n\n### Minikube\n\nMinikube is a tool that allows you to run Kubernetes locally on your computer.\n\nYou can find install instructions for minikube [here](https://github.com/kubernetes/minikube/releases). If you use homebrew the easiest thing to do is run this command:\n\n```bash\nbrew cask install minikube\n```\n\nYou can verify you have installed minikube by running:\n\n```bash\nminikube version\n```\n\n### Kubectl\n\nKubectl is the command-line tool for interfacing with Kubernetes. It allows you to deploy and manage your apps.\n\nInstallation instructions can be found [here](https://kubernetes.io/docs/tasks/tools/install-kubectl/). Once again our recommendation is to use homebrew for this:\n\n```bash\nbrew install kubernetes-cli\n```\n\nYou can verify you have installed kubectl by running:\n\n```bash\nkubectl version\n```\n\n## Getting Started\n\n1. Start your minikube by running:\n\n    ```bash\n    minikube start\n    ```\n\n2. Make sure your context is set to minikube.\n\n    ```bash\n    kubectl config get-contexts\n    ```\n\n    `minikube` should have a star next to it when you run this command. If it doesn't, you can set minikube to your context by running:\n\n    ```bash\n    kubectl config use-context minikube\n    ```","isFork":None},{"name":"kubernetes","url":"https://github.com/ekrengel/kubernetes","description":"Production-Grade Container Scheduling and Management","lang":["Python","Makefile","Shell","C","Go","Ruby","Nginx","SaltStack","PowerShell","Protocol Buffer","HTML"],"readme":"# Kubernetes\n\n[![Submit Queue Widget]][Submit Queue] [![GoDoc Widget]][GoDoc]\n\n<img src=\"https://github.com/kubernetes/kubernetes/raw/master/logo/logo.png\" width=\"100\">\n\n----\n\nKubernetes is an open source system for managing [containerized applications]\nacross multiple hosts, providing basic mechanisms for deployment, maintenance,\nand scaling of applications.\n\nKubernetes builds upon a decade and a half of experience at Google running\nproduction workloads at scale using a system called [Borg],\ncombined with best-of-breed ideas and practices from the community.\n\nKubernetes is hosted by the Cloud Native Computing Foundation ([CNCF]).\nIf you are a company that wants to help shape the evolution of\ntechnologies that are container-packaged, dynamically-scheduled\nand microservices-oriented, consider joining the CNCF.\nFor details about who's involved and how Kubernetes plays a role,\nread the CNCF [announcement].\n\n----\n\n## To start using Kubernetes\n\nSee our documentation on [kubernetes.io].\n\nTry our [interactive tutorial].\n\nTake a free course on [Scalable Microservices with Kubernetes].\n\n## To start developing Kubernetes\n\nThe [community repository] hosts all information about\nbuilding Kubernetes from source, how to contribute code\nand documentation, who to contact about what, etc.\n\nIf you want to build Kubernetes right away there are two options:\n\n##### You have a working [Go environment].\n\n```\n$ go get -d k8s.io/kubernetes\n$ cd $GOPATH/src/k8s.io/kubernetes\n$ make\n```\n\n##### You have a working [Docker environment].\n\n```\n$ git clone https://github.com/kubernetes/kubernetes\n$ cd kubernetes\n$ make quick-release\n```\n\nIf you are less impatient, head over to the [developer's documentation].\n\n## Support\n\nIf you need support, start with the [troubleshooting guide]\nand work your way through the process that we've outlined.\n\nThat said, if you have questions, reach out to us\n[one way or another][communication].\n\n[announcement]: https://cncf.io/news/announcement/2015/07/new-cloud-native-computing-foundation-drive-alignment-among-container\n[Borg]: https://research.google.com/pubs/pub43438.html\n[CNCF]: https://www.cncf.io/about\n[communication]: https://github.com/kubernetes/community/blob/master/communication.md\n[community repository]: https://github.com/kubernetes/community\n[containerized applications]: https://kubernetes.io/docs/concepts/overview/what-is-kubernetes/\n[developer's documentation]: https://github.com/kubernetes/community/tree/master/contributors/devel\n[Docker environment]: https://docs.docker.com/engine\n[Go environment]: https://golang.org/doc/install\n[GoDoc]: https://godoc.org/k8s.io/kubernetes\n[GoDoc Widget]: https://godoc.org/k8s.io/kubernetes?status.svg\n[interactive tutorial]: http://kubernetes.io/docs/tutorials/kubernetes-basics\n[kubernetes.io]: http://kubernetes.io\n[Scalable Microservices with Kubernetes]: https://www.udacity.com/course/scalable-microservices-with-kubernetes--ud615\n[Submit Queue]: http://submit-queue.k8s.io/#/ci\n[Submit Queue Widget]: http://submit-queue.k8s.io/health.svg?v=1\n[troubleshooting guide]: https://kubernetes.io/docs/tasks/debug-application-cluster/troubleshooting/ \n\n[![Analytics](https://kubernetes-site.appspot.com/UA-36037335-10/GitHub/README.md?pixel)]()\n","isFork":True},{"name":"corvus","url":"https://github.com/pavmohan/corvus","description":"Track and share your favorite cheeses","lang":["Python","CSS","JavaScript","C","HTML"],"readme":"# Corvus\nTrack and share your favorite cheeses","isFork":None},{"name":"cs290-assignment4-part1","url":"https://github.com/ekrengel/cs290-assignment4-part1","description":"Basic PHP Assignment","lang":["PHP"],"readme":None,"isFork":None},{"name":"cs290-assignment1","url":"https://github.com/ekrengel/cs290-assignment1","description":"CS290 Assignment 1 Into to Git","lang":[],"readme":None,"isFork":None},{"name":"271-Assignments","url":"https://github.com/ekrengel/271-Assignments","description":"Assembly Language Assignments","lang":["Assembly"],"readme":None,"isFork":None},{"name":"cs290-assignment3","url":"https://github.com/ekrengel/cs290-assignment3","description":"Basic JavaScript Assignment","lang":["JavaScript","CSS","HTML"],"readme":"Basic JavaScript Assignment\n===========================\n\nThis part of the assignment will cover fundamental JavaScript topics outside the context of web development. Fully automated tests will be provided for all assignment components. These are a tool to help you assess if your code is working as intended. Passing the tests does not guarantee a good grade nor does failing them guarantee a poor grade. The grade will be based on the extent to which you meet the requirements for code in this class and the extent to which the requirements in the source files are met.\n\nWork Log\n--------\nIn order to be eligible to receive an A you must submit a work log. It will not contribute points directly to the assignment, but without it the maximum grade you will be eligible for is an 89%. The purpose is to allow me to tune the difficulty of assignments to ensure that you are getting the amount of practice needed without spending too much time on any given topic.\n\nThe format is as follows:\n\n```\nStart: Wed, 7 Jan 2015 21:42:26 -0800\nEnd: Wed, 7 Jan 2015 22:42:26 -0800\nTasks: During this period I drew a diagram of the classes I will be using to implement the message passing portion of the assignment.\n\nStart: Fri, 9 Jan 2015 18:00:00 -0800\nEnd: Fri, 9 Jan 2015 20:42:26 -0800\nTasks: During this period I debugged for 1 hour to find a type and implement the send function of the cleint.\n```\n\nYou can get most of this information from the git log if you run:\n`git log --pretty=format:\"Start: %nEnd: %ad %nTasks: %s\" --date=rfc`\nYou just need to fill in the Start: time. Use the same date format. I would suggest copying and pasting the End time and just editing the time (or day if you went through midnight in a work session).\n\nHave one entry per work session.\n\nPut this log in log.txt.\n\nInstructions\n------------\n- Create a new repository called cs290-assignment3 at the location you specified in week 1.\n- Clone this repository and copy all of the contents to that repository\n  - The directory structure should look like this:\n    - cs290-assignment3\n      - log.txt\n      - qunit_runner.html\n      - qunit\n      - src\n      - tests\n- **You are only allowed to modify the files in the src directory.**\n  - There are some additional files, those are just a hook for us to eventually support a better testing system.\n  - The comments in the files in the src directory describe the intended behavior of the functions that you need to fill in. Please ask on the discussion boards if you are confused about the intended functionality of any function.\n  - Within those files, your code should go between the `//your code here` comments.\n  - You should replace `return undefined;` with appropriate return statements.\n- Add **cs290osu** as a collaborator. This will be an account that myself and the TAs share to access your repos.\n- To test your code open the `qunit_runner.html` web page locally. Along with the red errors check for statements saying something like `Expected 4 assertions, but 2 were run`. This means that the test had to abort early due to a error it could not recover from. That means there are two additional tests that are not even getting run.\n  - Given the nature of automated tests, it is easy to see what the expected values are and simply hard code those returns. If you do this for any portion of the assignment, you will get a 0 for the whole assignment. (For example, variableModification should work for any value, not just 42, but we are only testing the value 42. If you hardcode the values `47`, `'42'` and `'42foo'` that is a violation of this rule. )\n","isFork":None},{"name":"donneler.github.io","url":"https://github.com/ekrengel/donneler.github.io","description":None,"lang":["HTML","CSS","JavaScript"],"readme":None,"isFork":None},{"name":"cs290-assignment3p2","url":"https://github.com/ekrengel/cs290-assignment3p2","description":"Assignment 3 Part 2: Adding AJAX and HTML","lang":["JavaScript","HTML"],"readme":None,"isFork":None},{"name":"cs290-assignment2","url":"https://github.com/ekrengel/cs290-assignment2","description":"HTML and CSS","lang":["HTML","CSS"],"readme":None,"isFork":None},{"name":"CS161","url":"https://github.com/ekrengel/CS161","description":"Intro to Programming in C++","lang":["C++"],"readme":"CS161\n=====\n\nIntro to Programming in C++\n","isFork":None}]}, {"match":1,"linkedin_id":21396408,"gid":17517,"git_name":"Christian Berkhoff","git_login":"cberkhoff","git_location":"Seattle, WA","git_company":"Amazon LLC","git_org":[],"git_email":"christian.berkhoff@gmail.com","git_blog":None,"git_websiteUrl":None,"github_url":"https://github.com/cberkhoff","bio":None,"repos":[{"name":"howishell","url":"https://github.com/imella/howishell","description":"How I shell NASA Project","lang":["CoffeeScript","JavaScript"],"readme":"# How I Shell NASA Project\n\n* Daniel Aguilar\n* Christian Berkhoff\n* Joaquín Jaramillo\n* Ignacio Mella\n* Sebastían Rokas\n\n## Instalación\n\nEl único requisito es _node.js_\n\n    npm -g install handlebars coffee-script\n\n## Compilación\n\n    make\n\nEste comando compila `coffe/*.coffee` a `js/app.js` y `views\\*.hb` a `js/templates.js`\n\n# Moon Harvest\n\n## Overview\n\n### Do you have what's it takes to establish a colony on the Moon? What do you think it's best for the Moon?\n\nMan have always been fascinated with our natural satellite and there are a number of reasons to create a habitable or productive environment there.\n\nMoon Harvest is a game where you take the role of a Chief Operating Officer in charge of the Moon Colony Mission. \n\nThis game is powered by a complex (still in development) moon environment simulator. This complexity along with the hosts variables involved should create some awareness of how hard your quest will be.\n\nYour final goal is to create a sustainable space industry on the surface of the Moon using all resources available.\n\nYour main objetives will be:\n  - Create a sustainable process in terms of Energy.\n  - Install observatory instruments to take Astronomy to the next level.\n  - Install defensive mechanisms to protect your colony and Earth from Meteor Showers.\n  - And finally send living beings and Humans from Earth to test the environment you've created.\n\nBuild a Robot Factory, extract moon rocks, retrieve other resources such as Alumina, Silica and  He-3 to create other buildings that will help you through your journey.\n\nDuring this mission you'll get the counsel of five advisors that will guide you to achieve your goal. Each one with his own perspective of what's best. Which one will you follow?\n\n## Think, build, become the first man to successfully establish a colony on the Moon and have lots of Fun ! That's what Moon Harvest has for you!\n ","isFork":None},{"name":"buho","url":"https://github.com/cberkhoff/buho","description":None,"lang":[],"readme":"# buho","isFork":None},{"name":"mine.js","url":"https://github.com/cberkhoff/mine.js","description":"A simple game concept written in coffeescript","lang":["CoffeeScript","JavaScript"],"readme":None,"isFork":None},{"name":"winlepp","url":"https://github.com/cberkhoff/winlepp","description":"Winlepp was created as a part of a course at the Universidad de Chile. The application is focused in parallel triangle mesh refinement algorithms. It is written in C++ using QT Framework and the Eclipse IDE","lang":["Prolog","C++"],"readme":"winlepp\n=======\n\nWinlepp was created as a part of a course at the Universidad de Chile. The application is focused in parallel triangle mesh refinement algorithms. It is written in C++ using QT Framework and the Eclipse IDE","isFork":None},{"name":"android-location-picker","url":"https://github.com/cberkhoff/android-location-picker","description":"Dynamic Android control which first shows a list of countries and then, depending on the selected values, shows the corresponding child regions using dropdowns. Contains Chilean regional subdivisions. Uses Gradle.","lang":["Groovy","Shell","Java"],"readme":"android-location-picker\n=======================\n\nDynamic Android control which first shows a list of countries and then, depending on the selected values, shows the corresponding child regions using dropdowns. Contains Chilean regional subdivisions. Uses Gradle.\n","isFork":None},{"name":"facebook-android-sdk","url":"https://github.com/cberkhoff/facebook-android-sdk","description":"Facebook SDK for Android","lang":["Groovy","Java","Shell"],"readme":None,"isFork":True},{"name":"blackjack","url":"https://github.com/cberkhoff/blackjack","description":"Clojure example for an introductory worshop. It's the classic carioca card game.","lang":["Clojure"],"readme":"Blackjack\n=========\n\nClojure example for an introductory worshop. It's a simplification of the classic Blackjack card game.\n\nFor a game explanation go to [this links](http://www.pagat.com/banking/blackjack.html).\n","isFork":None},{"name":"hunchator","url":"https://github.com/imella/hunchator","description":None,"lang":["Ruby","CoffeeScript","JavaScript"],"readme":None,"isFork":None},{"name":"team11","url":"https://github.com/spaceappchile/team11","description":None,"lang":["CoffeeScript","JavaScript"],"readme":"# How I Shell NASA Project\n\n* Daniel Aguilar\n* Christian Berkhoff\n* Joaquín Jaramillo\n* Ignacio Mella\n* Sebastían Rokas\n\n## Instalación\n\nEl único requisito es _node.js_\n\n    npm -g install handlebars coffee-script\n\n## Compilación\n\n    make\n\nEste comando compila `coffe/*.coffee` a `js/app.js` y `views\\*.hb` a `js/templates.js`\n\n# Moon Harvest\n\n## Overview\n\n### Do you have what's it takes to establish a colony on the Moon? What do you think it's best for the Moon?\n\nMan have always been fascinated with our natural satellite and there are a number of reasons to create a habitable or productive environment there.\n\nMoon Harvest is a game where you take the role of a Chief Operating Officer in charge of the Moon Colony Mission. \n\nThis game is powered by a complex (still in development) moon environment simulator. This complexity along with the hosts variables involved should create some awareness of how hard your quest will be.\n\nYour final goal is to create a sustainable space industry on the surface of the Moon using all resources available.\n\nYour main objetives will be:\n  - Create a sustainable process in terms of Energy.\n  - Install observatory instruments to take Astronomy to the next level.\n  - Install defensive mechanisms to protect your colony and Earth from Meteor Showers.\n  - And finally send living beings and Humans from Earth to test the environment you've created.\n\nBuild a Robot Factory, extract moon rocks, retrieve other resources such as Alumina, Silica and  He-3 to create other buildings that will help you through your journey.\n\nDuring this mission you'll get the counsel of five advisors that will guide you to achieve your goal. Each one with his own perspective of what's best. Which one will you follow?\n\n## Think, build, become the first man to successfully establish a colony on the Moon and have lots of Fun ! That's what Moon Harvest has for you!\n ","isFork":None},{"name":"schnaps","url":"https://github.com/imella/schnaps","description":None,"lang":["CoffeeScript","JavaScript"],"readme":"# Schnaps\n\nSchaps, what could I drink tonight?","isFork":None},{"name":"euler","url":"https://github.com/cberkhoff/euler","description":"Solution to ProjectEuler problems using Java","lang":["Java"],"readme":"euler\n=====\n\nSolution to ProjectEuler problems using Java\n","isFork":None},{"name":"cc51s","url":"https://github.com/cberkhoff/cc51s","description":"A simple game written in Java using RMI for the CC51S course","lang":["Java"],"readme":"cc51s\n=====\n\nA simple game written in Java using RMI for the CC51S course","isFork":None},{"name":"UrlImageViewHelper","url":"https://github.com/cberkhoff/UrlImageViewHelper","description":"Android library that sets an ImageView's contents from a url. Manages image downloading, caching, and makes your coffee too.","lang":["Groovy","Java"],"readme":"## UrlImageViewHelper\nUrlImageViewHelper will fill an ImageView with an image that is found at a URL.\n\n### Sample Project\n\nThe sample will do a Google Image Search and load/show the results asynchronously.\n\n![](https://raw.github.com/koush/UrlImageViewHelper/master/helper2.png)\n\n### Download\n\nDownload [the latest JAR](http://repository.sonatype.org/service/local/artifact/maven/redirect?r=central-proxy&g=com.koushikdutta.urlimageviewhelper&a=urlimageviewhelper&v=LATEST\n) or grab via Maven:\n\n```xml\n<dependency>\n    <groupId>com.koushikdutta.urlimageviewhelper</groupId>\n    <artifactId>urlimageviewhelper</artifactId>\n    <version>(insert latest version)</version>\n</dependency>\n```\n\n### Usage\n\nUrlImageViewHelper will automatically download and manage all the web images and ImageViews.\nDuplicate urls will not be loaded into memory twice. Bitmap memory is managed by using\na weak reference hash table, so as soon as the image is no longer used by you,\nit will be garbage collected automatically.\n\nUsage is simple:\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\");\n```\n\n\nWant a placeholder image while it is being downloaded?\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", R.drawable.placeholder);\n```\n\n\nDon't want to use a placeholder resource, but a drawable instead?\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", drawable);\n```\n\n\nWhat if you want to preload images for snazzy fast loading?\n\n```java\nUrlImageViewHelper.loadUrlDrawable(context, \"http://example.com/image.png\");\n```\n\n\nWhat if you only want to cache the images for a minute?\n\n```java\n// Note that the 3rd argument \"null\" is an optional interstitial\n// placeholder image.\nUrlImageViewHelper.setUrlDrawable(imageView, \"http://example.com/image.png\", null, 60000);\n```\n\nUrlImageViewHelper is pretty smart. It can even load the photo for an Android contact\nif given a Contact Content Provider URI.\n\n```java\nUrlImageViewHelper.setUrlDrawable(imageView, \"content://com.android.contacts/contacts/1115\", R.drawable.dummy_contact_photo);\n```\n\n### FAQ\n\n**Does it work in list adapters when views are reused? (convertView)**\n\nYes.\n\n\n### Featured Implementations\n\n * [ROM Manager](https://play.google.com/store/apps/details?id=com.koushikdutta.rommanager&hl=en)\n * [Carbon](https://play.google.com/store/apps/details?id=com.koushikdutta.backup&hl=en)\n * Let me know if you use this library, so I can add it to the list!\n","isFork":True},{"name":"ActionBarSherlock","url":"https://github.com/cberkhoff/ActionBarSherlock","description":"Library for implementing the action bar design pattern using the native action bar on Android 4.0+ and a custom implementation on pre-4.0 through a single API and theme.","lang":[],"readme":"ActionBarSherlock\n=================\n\nActionBarSherlock is an standalone library designed to facilitate the use of\nthe action bar design pattern across all versions of Android through a single\nAPI.\n\nThe library will automatically use the [native ActionBar][2] implementation on\nAndroid 4.0 or later. For previous versions which do not include ActionBar, a\ncustom action bar implementation based on the sources of Ice Cream Sandwich\nwill automatically be wrapped around the layout. This allows you to easily\ndevelop an application with an action bar for every version of Android from 2.x\nand up.\n\n**See http://actionbarsherlock.com for more information.**\n\n![Example Image][3]\n\nTry out the sample applications on the Android Market: [Feature Demos][4],\n[Fragments][5], and [RoboGuice][6].\n\nContinuous integration is provided by [Travis CI][7].\n\n\n\nDeveloped By\n============\n\n* Jake Wharton - <jakewharton@gmail.com>\n\n\n\nLicense\n=======\n\n    Copyright 2012 Jake Wharton\n\n    Licensed under the Apache License, Version 2.0 (the \"License\");\n    you may not use this file except in compliance with the License.\n    You may obtain a copy of the License at\n\n       http://www.apache.org/licenses/LICENSE-2.0\n\n    Unless required by applicable law or agreed to in writing, software\n    distributed under the License is distributed on an \"AS IS\" BASIS,\n    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n    See the License for the specific language governing permissions and\n    limitations under the License.\n\n\n\n\n\n [1]: http://android-developers.blogspot.com/2011/03/fragments-for-all.html\n [2]: http://developer.android.com/guide/topics/ui/actionbar.html\n [3]: http://actionbarsherlock.com/static/feature.png\n [4]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.demos\n [5]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.fragments\n [6]: https://play.google.com/store/apps/details?id=com.actionbarsherlock.sample.roboguice\n [7]: https://travis-ci.org/JakeWharton/ActionBarSherlock\n","isFork":True},{"name":"emacs-clojure-vagrant","url":"https://github.com/cberkhoff/emacs-clojure-vagrant","description":"A simple dev environment for clojure.","lang":["Ruby","Shell","Emacs Lisp"],"readme":"Vagrant setup for Clojure\n=========================\n\nStrated as a simple fork from [https://github.com/Seajure/emacs-clojure-vagrant](Seajure) but ended being a completedly different thing.\n\nSetup\n=====\n   - Install Vagrant following the instructions [http://vagrantup.com/docs/getting-started/index.html](Vagrant Getting Started).\n   - Download this\n   - `vagrant up` inside the directory containing this readme\n\nUsage\n-----\n\nContents\n--------\n- Java\n- Git\n- Leiningen\n- Emacs\n- Postgresql","isFork":True},{"name":"node-vagrant","url":"https://github.com/cberkhoff/node-vagrant","description":"A quick and dirty node setup using vagrant and chef","lang":["Ruby"],"readme":None,"isFork":True},{"name":"express-lingua","url":"https://github.com/cberkhoff/express-lingua","description":"A i18n middleware for the Express.js framework.","lang":["JavaScript"],"readme":"# Lingua and Lengua\n\n[Lingua](https://github.com/akoenig/express-lingua) is a middleware for the Express.js framework that helps you to internationalise your webapp easily. It determines the language of the user agent and pushes the i18n resources to your views.\n\nLengua is a fork form Lingua which introduces some features.\n\n## Installation\n\n    $ npm install lengua\n\n## Show me the code!\n\nTo translate somenthig (Coffeekup)\n\n<pre>\n@t 'Name'\n@t 'navigation.german'\n</pre>\n\nTo get the current country code, language and locale respectively\n\n<pre>\n@currentCountry\n@currentCountryLanguage\n@currentLocale\n</pre>\n\n## Added features\n\n### For the common mortal\n\n1. User locale stored in session instead of cookie\n\n2. Not case sensitive\n\n3. Country Code\n\n4. If the current locale (e.g. en-US) doesnt define a key, Lengua does a lookup in the global locale (e.g. en).\n\n### Other Stuff\n\n1. Tests :)\n\n## TODO\n\n1. Locale storage as strategy (Cookie, Session, Azure, etc).\n\n2. Cache update (when all fallbacks fail).\n\n## Dev\n\n### Tests\n\n<pre>\nnpm install jasmine-node\ncd spec/\njasmine-node.cmd .\\guru.spec.js\njasmine-node.cmd .\\lingua.spec.js\n</pre>\n\n## License\n\n[MIT License](http://www.opensource.org/licenses/mit-license.php)\n\n## Author\n\nCopyright (c) 2011,\n \n[André König](http://lochkartenstanzer.de) ([Google+](http://profile.lochkartenstanzer.de)) (andre.koenig -[at]- gmail [*dot*] com)\n\n[Christian Berkhoff](#) (christian.berkhoff - [at] - gmail [#dot#] com)\n","isFork":True},{"name":"paperclipdb","url":"https://github.com/cberkhoff/paperclipdb","description":"Database storage support for paperclip file attachment plugin.","lang":["Ruby"],"readme":None,"isFork":True}]}
,{"match":1,"linkedin_id":213954281,"gid":15986,"git_name":"Erin Krengel","git_login":"ekrengel","git_location":"Seattle, WA","git_company":None,"git_org":[],"git_email":None,"git_blog":None,"git_websiteUrl":None,"github_url":"https://github.com/ekrengel","bio":None,"repos":[{"name":"introk8s","url":"https://github.com/ekrengel/introk8s","description":"ACT-W Introduction to K8s","lang":["Dockerfile","Go"],"readme":"# ACT-W Introduction to K8s\n\nThis workshop was developed for ACT-W Seattle 2018.\n\n## Pre-Workshop Installations\n\nThere is a number of open-source software you will need to install to follow along with the workshop.\n\n### Minikube\n\nMinikube is a tool that allows you to run Kubernetes locally on your computer.\n\nYou can find install instructions for minikube [here](https://github.com/kubernetes/minikube/releases). If you use homebrew the easiest thing to do is run this command:\n\n```bash\nbrew cask install minikube\n```\n\nYou can verify you have installed minikube by running:\n\n```bash\nminikube version\n```\n\n### Kubectl\n\nKubectl is the command-line tool for interfacing with Kubernetes. It allows you to deploy and manage your apps.\n\nInstallation instructions can be found [here](https://kubernetes.io/docs/tasks/tools/install-kubectl/). Once again our recommendation is to use homebrew for this:\n\n```bash\nbrew install kubernetes-cli\n```\n\nYou can verify you have installed kubectl by running:\n\n```bash\nkubectl version\n```\n\n## Getting Started\n\n1. Start your minikube by running:\n\n    ```bash\n    minikube start\n    ```\n\n2. Make sure your context is set to minikube.\n\n    ```bash\n    kubectl config get-contexts\n    ```\n\n    `minikube` should have a star next to it when you run this command. If it doesn't, you can set minikube to your context by running:\n\n    ```bash\n    kubectl config use-context minikube\n    ```","isFork":None},{"name":"kubernetes","url":"https://github.com/ekrengel/kubernetes","description":"Production-Grade Container Scheduling and Management","lang":["Python","Makefile","Shell","C","Go","Ruby","Nginx","SaltStack","PowerShell","Protocol Buffer","HTML"],"readme":"# Kubernetes\n\n[![Submit Queue Widget]][Submit Queue] [![GoDoc Widget]][GoDoc]\n\n<img src=\"https://github.com/kubernetes/kubernetes/raw/master/logo/logo.png\" width=\"100\">\n\n----\n\nKubernetes is an open source system for managing [containerized applications]\nacross multiple hosts, providing basic mechanisms for deployment, maintenance,\nand scaling of applications.\n\nKubernetes builds upon a decade and a half of experience at Google running\nproduction workloads at scale using a system called [Borg],\ncombined with best-of-breed ideas and practices from the community.\n\nKubernetes is hosted by the Cloud Native Computing Foundation ([CNCF]).\nIf you are a company that wants to help shape the evolution of\ntechnologies that are container-packaged, dynamically-scheduled\nand microservices-oriented, consider joining the CNCF.\nFor details about who's involved and how Kubernetes plays a role,\nread the CNCF [announcement].\n\n----\n\n## To start using Kubernetes\n\nSee our documentation on [kubernetes.io].\n\nTry our [interactive tutorial].\n\nTake a free course on [Scalable Microservices with Kubernetes].\n\n## To start developing Kubernetes\n\nThe [community repository] hosts all information about\nbuilding Kubernetes from source, how to contribute code\nand documentation, who to contact about what, etc.\n\nIf you want to build Kubernetes right away there are two options:\n\n##### You have a working [Go environment].\n\n```\n$ go get -d k8s.io/kubernetes\n$ cd $GOPATH/src/k8s.io/kubernetes\n$ make\n```\n\n##### You have a working [Docker environment].\n\n```\n$ git clone https://github.com/kubernetes/kubernetes\n$ cd kubernetes\n$ make quick-release\n```\n\nIf you are less impatient, head over to the [developer's documentation].\n\n## Support\n\nIf you need support, start with the [troubleshooting guide]\nand work your way through the process that we've outlined.\n\nThat said, if you have questions, reach out to us\n[one way or another][communication].\n\n[announcement]: https://cncf.io/news/announcement/2015/07/new-cloud-native-computing-foundation-drive-alignment-among-container\n[Borg]: https://research.google.com/pubs/pub43438.html\n[CNCF]: https://www.cncf.io/about\n[communication]: https://github.com/kubernetes/community/blob/master/communication.md\n[community repository]: https://github.com/kubernetes/community\n[containerized applications]: https://kubernetes.io/docs/concepts/overview/what-is-kubernetes/\n[developer's documentation]: https://github.com/kubernetes/community/tree/master/contributors/devel\n[Docker environment]: https://docs.docker.com/engine\n[Go environment]: https://golang.org/doc/install\n[GoDoc]: https://godoc.org/k8s.io/kubernetes\n[GoDoc Widget]: https://godoc.org/k8s.io/kubernetes?status.svg\n[interactive tutorial]: http://kubernetes.io/docs/tutorials/kubernetes-basics\n[kubernetes.io]: http://kubernetes.io\n[Scalable Microservices with Kubernetes]: https://www.udacity.com/course/scalable-microservices-with-kubernetes--ud615\n[Submit Queue]: http://submit-queue.k8s.io/#/ci\n[Submit Queue Widget]: http://submit-queue.k8s.io/health.svg?v=1\n[troubleshooting guide]: https://kubernetes.io/docs/tasks/debug-application-cluster/troubleshooting/ \n\n[![Analytics](https://kubernetes-site.appspot.com/UA-36037335-10/GitHub/README.md?pixel)]()\n","isFork":True},{"name":"corvus","url":"https://github.com/pavmohan/corvus","description":"Track and share your favorite cheeses","lang":["Python","CSS","JavaScript","C","HTML"],"readme":"# Corvus\nTrack and share your favorite cheeses","isFork":None},{"name":"cs290-assignment4-part1","url":"https://github.com/ekrengel/cs290-assignment4-part1","description":"Basic PHP Assignment","lang":["PHP"],"readme":None,"isFork":None},{"name":"cs290-assignment1","url":"https://github.com/ekrengel/cs290-assignment1","description":"CS290 Assignment 1 Into to Git","lang":[],"readme":None,"isFork":None},{"name":"271-Assignments","url":"https://github.com/ekrengel/271-Assignments","description":"Assembly Language Assignments","lang":["Assembly"],"readme":None,"isFork":None},{"name":"cs290-assignment3","url":"https://github.com/ekrengel/cs290-assignment3","description":"Basic JavaScript Assignment","lang":["JavaScript","CSS","HTML"],"readme":"Basic JavaScript Assignment\n===========================\n\nThis part of the assignment will cover fundamental JavaScript topics outside the context of web development. Fully automated tests will be provided for all assignment components. These are a tool to help you assess if your code is working as intended. Passing the tests does not guarantee a good grade nor does failing them guarantee a poor grade. The grade will be based on the extent to which you meet the requirements for code in this class and the extent to which the requirements in the source files are met.\n\nWork Log\n--------\nIn order to be eligible to receive an A you must submit a work log. It will not contribute points directly to the assignment, but without it the maximum grade you will be eligible for is an 89%. The purpose is to allow me to tune the difficulty of assignments to ensure that you are getting the amount of practice needed without spending too much time on any given topic.\n\nThe format is as follows:\n\n```\nStart: Wed, 7 Jan 2015 21:42:26 -0800\nEnd: Wed, 7 Jan 2015 22:42:26 -0800\nTasks: During this period I drew a diagram of the classes I will be using to implement the message passing portion of the assignment.\n\nStart: Fri, 9 Jan 2015 18:00:00 -0800\nEnd: Fri, 9 Jan 2015 20:42:26 -0800\nTasks: During this period I debugged for 1 hour to find a type and implement the send function of the cleint.\n```\n\nYou can get most of this information from the git log if you run:\n`git log --pretty=format:\"Start: %nEnd: %ad %nTasks: %s\" --date=rfc`\nYou just need to fill in the Start: time. Use the same date format. I would suggest copying and pasting the End time and just editing the time (or day if you went through midnight in a work session).\n\nHave one entry per work session.\n\nPut this log in log.txt.\n\nInstructions\n------------\n- Create a new repository called cs290-assignment3 at the location you specified in week 1.\n- Clone this repository and copy all of the contents to that repository\n  - The directory structure should look like this:\n    - cs290-assignment3\n      - log.txt\n      - qunit_runner.html\n      - qunit\n      - src\n      - tests\n- **You are only allowed to modify the files in the src directory.**\n  - There are some additional files, those are just a hook for us to eventually support a better testing system.\n  - The comments in the files in the src directory describe the intended behavior of the functions that you need to fill in. Please ask on the discussion boards if you are confused about the intended functionality of any function.\n  - Within those files, your code should go between the `//your code here` comments.\n  - You should replace `return undefined;` with appropriate return statements.\n- Add **cs290osu** as a collaborator. This will be an account that myself and the TAs share to access your repos.\n- To test your code open the `qunit_runner.html` web page locally. Along with the red errors check for statements saying something like `Expected 4 assertions, but 2 were run`. This means that the test had to abort early due to a error it could not recover from. That means there are two additional tests that are not even getting run.\n  - Given the nature of automated tests, it is easy to see what the expected values are and simply hard code those returns. If you do this for any portion of the assignment, you will get a 0 for the whole assignment. (For example, variableModification should work for any value, not just 42, but we are only testing the value 42. If you hardcode the values `47`, `'42'` and `'42foo'` that is a violation of this rule. )\n","isFork":None},{"name":"donneler.github.io","url":"https://github.com/ekrengel/donneler.github.io","description":None,"lang":["HTML","CSS","JavaScript"],"readme":None,"isFork":None},{"name":"cs290-assignment3p2","url":"https://github.com/ekrengel/cs290-assignment3p2","description":"Assignment 3 Part 2: Adding AJAX and HTML","lang":["JavaScript","HTML"],"readme":None,"isFork":None},{"name":"cs290-assignment2","url":"https://github.com/ekrengel/cs290-assignment2","description":"HTML and CSS","lang":["HTML","CSS"],"readme":None,"isFork":None},{"name":"CS161","url":"https://github.com/ekrengel/CS161","description":"Intro to Programming in C++","lang":["C++"],"readme":"CS161\n=====\n\nIntro to Programming in C++\n","isFork":None}]}]

In [ ]:
git  = json.dumps(git)

In [ ]:
test = json.dumps(test)

In [ ]:

from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, ArrayType, BooleanType

GitSchema = StructType([
        StructField("repos",ArrayType(StructType([StructField("lang", ArrayType(StringType())),StructField("isFork", BooleanType(), False),StructField("description", StringType(), False),StructField("name", StringType(), False),StructField("readme", StringType(), False), StructField("url", StringType(), False)]))),
        StructField("git_org", ArrayType(StringType())),
        StructField("git_name", StringType()),
        StructField("bio", StringType()),
        StructField("git_location", StringType()),
        StructField("git_company", StringType()),
        StructField("gid", IntegerType()),
        StructField("github_url", StringType()),
        StructField("git_email", StringType()),
        StructField("git_blog", StringType()),      
        StructField("git_login",StringType()),
        StructField("git_websiteUrl", StringType()),
        StructField("linkedin_id", IntegerType())])

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, ArrayType

LinkSchema = StructType([
        StructField("education",ArrayType(StructType([StructField("major", StringType(), False),StructField("name", StringType(), False), StructField("summary", StringType(), False)]))),
        StructField("experience", ArrayType(StructType([StructField("description", StringType(), False),StructField("organization", StringType(), False), StructField("title", StringType(), False)]))),
        StructField("full_name", StringType()),
        StructField("headline", StringType()),
        StructField("industry", StringType()),
        StructField("interests", ArrayType(StringType())),
        StructField("linkedin", IntegerType()),
        StructField("linkedin_url", StringType()),
        StructField("location", StringType()),
        StructField("projects", ArrayType(StructType([StructField("title", StringType(), False), StructField("description", StringType(), False)]))),
        StructField("publications",ArrayType(StructType([StructField("name", StringType(), False), StructField("summary", StringType(), False)]))),
        StructField("skills", ArrayType(StringType())),
        StructField("summary", StringType()),
        StructField("websites",  ArrayType(StructType([StructField("url", StringType(), False), StructField("description", StringType(), False)])))    
   ])

In [ ]:
rddLink = sc.parallelize([test])

In [ ]:
rddGit = sc.parallelize([git])

In [ ]:
link = sqlContext.read.json(rddLink, LinkSchema)

In [ ]:
link.printSchema()

In [ ]:
link = link.withColumnRenamed('linkedin', 'linkedin_id')

In [ ]:
link = link.withColumn('skills', F.concat_ws(',',link.skills))

In [ ]:
link = link.withColumn('edu_name', F.concat_ws(',', F.col('education.name')))

link = link.withColumn('exp_org', F.concat_ws(',', F.col('experience.organization')))

link = link.withColumn('website', F.concat_ws(',', F.col('websites.url')))

In [ ]:
link = link.withColumn('_pro_pub_title', F.concat_ws(',', F.col('projects.title'), F.col('publications.name')))

In [ ]:
link = link.withColumn('_pro_pub_desc', F.concat_ws(',', F.col('projects.description'), F.col('publications.summary')))

In [ ]:
link = link.withColumn("exp_desc", F.concat_ws(',', F.col('experience.description')))

In [ ]:
link = link.withColumn('_skills', F.concat_ws(',', F.col('skills'), F.col('projects.title'), F.col('publications.name')))

In [ ]:
link = link.withColumn('_edu_exp', F.concat_ws(',', F.col('education.name'), F.col('experience.organization')))

In [ ]:
link = link.withColumn('id', F.col('linkedin_id').cast('string'))

In [ ]:
link.select('id').show(2, False)

In [ ]:
link.select('skills','edu_name','exp_org').show(1, False)

In [ ]:
link.toPandas()

In [ ]:
git = sqlContext.read.json(rddGit, GitSchema)

In [ ]:
git.printSchema()

In [ ]:
git = git.withColumn('git_org', F.concat_ws(',',F.col('git_org')))

git = git.withColumn('_pro_pub_title', F.concat_ws(',', F.col('repos.name')))

git = git.withColumn('_pro_pub_desc', F.concat_ws(',', F.col('repos.description')))

git = git.withColumn('git_lang', F.col('repos.lang').cast("string"))

git = git.withColumn('_skills', F.concat_ws(',', F.col('git_lang'),F.col('_pro_pub_title')))

In [ ]:
git.toPandas()

In [ ]:
git = git.withColumn('summary', F.concat_ws(',', F.col('_pro_pub_title'),F.col('git_email'), F.col('bio'), F.col('git_company')))

In [ ]:
git = git.withColumn('id', F.col('gid').cast("string"))

In [ ]:
git = git.withColumn('id', F.regexp_replace(F.col('id'), '([\d]+)', "g$1"))

In [ ]:
git.select('summary').show(3, False)

In [ ]:
git.select('_skills').show(3, False)

In [ ]:
model_TFIDF =PipelineModel.load("wasb://test-container@tfsmodelstorage.blob.core.windows.net/model_tfidf0913")

In [ ]:

data_join = git.select('id','git_lang','_skills', git.bio.alias('_edu_exp'), 'bio', git._pro_pub_desc.alias('exp_desc'), 'summary','_pro_pub_title','_pro_pub_desc').union(link.select('id',link.skills.alias('git_lang'),'_skills','_edu_exp',link.headline.alias('bio'), 'exp_desc', 'summary','_pro_pub_title','_pro_pub_desc'))
data_join = data_join.na.fill('')

In [ ]:
data_feature = model_TFIDF.transform(data_join)


In [ ]:
data_feature = data_feature.select('id', 'git_langIDF','_skillsIDF', '_edu_expIDF','bioIDF', 'exp_descIDF','summaryIDF','_pro_pub_titleIDF','_pro_pub_descIDF')


In [ ]:
lookupTable = sc.broadcast(data_feature.rdd.map(lambda x: (x['id'], 
                                                  {'git_langIDF':x['git_langIDF'],
                                                   '_skillsIDF':x['_skillsIDF'],
                                                   '_edu_expIDF':x['_edu_expIDF'],
                                                   'bioIDF':x['bioIDF'],
                                                   'exp_descIDF':x['exp_descIDF'],
                                                   'summaryIDF':x['summaryIDF'],
                                                   '_pro_pub_titleIDF':x['_pro_pub_titleIDF'],
                                                   '_pro_pub_descIDF':x['_pro_pub_descIDF']})).collectAsMap())

In [ ]:
joined = git.select('gid', 'linkedin_id', 'git_login','git_name','git_location','git_company','git_blog','github_url','git_websiteUrl').join(link.select('linkedin_id','full_name','location','edu_name','exp_org','linkedin_url',link.websites.url.alias('website')), ['linkedin_id'])

In [ ]:
count = joined.cube("linkedin_id").count()

In [ ]:
joined = joined.join(count,['linkedin_id'])


In [ ]:
joined = joined.withColumn('gid', F.regexp_replace(F.col('gid'), '([\d]+)', 'g$1'))

In [ ]:
joined = joined.withColumn('linkedin_id', F.col('linkedin_id').cast('string'))

In [ ]:
def cosine_similarity(X, Y):
    denom = X.norm(2) * Y.norm(2)
    if denom == 0.0:
        return -1.0
    else:
        return X.dot(Y)*1. / float(denom)

def similarities(linkedin_id, gid, lookupTable):
    X, Y = lookupTable.value[linkedin_id], lookupTable.value[gid]
    gitlang_simi = cosine_similarity(X['git_langIDF'], Y['git_langIDF'])
    skill_simi = cosine_similarity(X['_skillsIDF'], Y['_skillsIDF'])
    edu_exp_simi = cosine_similarity(X['_edu_expIDF'], Y['_edu_expIDF'])
    bio_simi = cosine_similarity(X['bioIDF'], Y['bioIDF'])
    exp_desc_simi = cosine_similarity(X['exp_descIDF'], Y['exp_descIDF'])
    summary_simi = cosine_similarity(X['summaryIDF'], Y['summaryIDF'])
    pro_pub_title_simi = cosine_similarity(X['_pro_pub_titleIDF'], Y['_pro_pub_titleIDF'])
    pro_pub_desc_simi = cosine_similarity(X['_pro_pub_descIDF'], Y['_pro_pub_descIDF'])
    return gitlang_simi, skill_simi, edu_exp_simi, bio_simi, exp_desc_simi, summary_simi, pro_pub_title_simi, pro_pub_desc_simi

In [ ]:
measureMapping.printSchema()

In [ ]:
df = joined.toPandas()

In [ ]:
df

In [ ]:
def preprocess(column):
    from name_tools import split
    
    if column is None:
        return ''
    else: 
        m = split(column)
        if(m[1]==''):
            full_name = m[2];
        elif(m[2]==''):
            full_name = m[1];
        else:
            full_name = ' '.join((m[1], m[2]));
        return full_name

In [ ]:
def doublemetaphone(str1, str2):
    # compute the jaro-winkler distance between first names and last names's double metaphone codes
    # equal weights are assigned to the first and last name
    # returns null when either string is null 
    
    
    if(str1 is None or str1 is None):
        return ''
    else: 
        weight1 = 0.5
        weight2 = 0.5
        
        import re
        from name_tools import split
        str1 = re.sub(r'[^\x00-\x7f]',r' ',str1)   # Each char is a Unicode codepoint.
        str2 = re.sub(r'[^\x00-\x7f]',r' ',str2)
        m1 = split(str1) #(prefix, firstname, lastname, surffix)
        m2 = split(str2)  
   
        dm1 = phonetics.dmetaphone(m1[1].encode('ascii').decode('ascii'))+ phonetics.dmetaphone(m1[2].encode('ascii').decode('ascii'))
        dm2 = phonetics.dmetaphone(m2[1].encode('ascii').decode('ascii'))+ phonetics.dmetaphone(m2[2].encode('ascii').decode('ascii'))
     
    
        dm_first1 =dm1[0]+dm1[1]
        dm_last1 = dm1[2]+dm1[3]
    
        dm_first2 = dm2[0]+dm2[1]
        dm_last2 = dm2[2]+dm2[3]
        
        
        if(m2[1] ==''):
            d = jaro_winkler(unicode(dm_first1), unicode(dm_last2))*weight1+ jaro_winkler(unicode(dm_last1), unicode(dm_first2))*weight2
        elif(m1[1] == ''):
            d = jaro_winkler(unicode(dm_first2),unicode( dm_last1))*weight1+ jaro_winkler(unicode(dm_last2), unicode(dm_first1))*weight2      
        else:
            d = jaro_winkler(unicode(dm_first1),unicode(dm_first2))*weight1+ jaro_winkler(unicode(dm_last1), unicode(dm_last2))*weight2
        return round(d,5)

In [ ]:
def fuzz_sort(str1, str2):
    
    result = [];
    if(str1 ==[] or str2==[]):
        return 0
    else:
        if(isinstance(str1, list)):
            if(isinstance(str2, list)):
                for value in itertools.product(str1, str2):
                   #print(value)
                    result.append(fuzz.token_sort_ratio(preprocess_company(value[0]), preprocess_company(value[1])))
            #print("list1")
            #print(result)
                d = max(result);
            else:
                for value in enumerate(str1):
                    result.append(fuzz.token_sort_ratio(preprocess_company(value[1]), preprocess_company(str2)))
            #print("notlist2")
            #print(result)
                d = max(result)   
        elif(isinstance(str2, list)):
            for value in enumerate(str2):
                result.append(fuzz.token_sort_ratio(preprocess_company(value[1]), preprocess_company(str1)))
        #print("list2")
        #print(result)
            d = max(result);
        else:
            d = fuzz.token_sort_ratio(str1, str2)
        return(d)

In [ ]:
def leven_list(str1, str2):
    result = [];
    if(str1 ==[] or str2==[] or str1 is None or str2 is None):
    	return None
    else:
    	if(isinstance(str1, list)):
    		if(isinstance(str2, list)):
    			for value in itertools.product(str1, str2):
    				result.append(ld(unicode(value[0].lower()), unicode(value[1].lower()))*1./max(len(unicode(value[0])),len(unicode(value[1]))))
    			d = min(result);
    		else:
    			for value in enumerate(str1):
    				result.append(ld(unicode(value[1].lower()), unicode(str2.lower()))*1./max(len(unicode(value[1])),len(unicode(str2))))
    				d = min(result)
    	elif(isinstance(str2, list)):
    		for value in enumerate(str2):
    			result.append(ld(unicode(value[1].lower()), unicode(str1.lower()))*1./max(len(unicode(value[1])),len(unicode(str1))))
    		d = min(result);
    	else:
    		d = ld(unicode(str1), unicode(str2))*1./max(len(unicode(str1)),len(unicode(str2)))
    	return(d)

In [ ]:
def preprocess_company(column):
    # return a cleaned-up company name
    # first convert all words into lower case
    # remove "@", "," "ltd.", 'llc.' and 'co.' 
    
    
    if column is None:
        return ''
    else:
        lower = column.lower()
        result = re.sub(r"\,|@|\.", " ", lower)
        result = re.sub(' +',' ', result).strip()
        for word in ['llc','ltd','co','inc']:
            patt = re.compile('(\s*)'+word+'(\s*)')
            result = patt.sub('', result)
        return result


In [ ]:
df['git_company'] = df.apply(lambda row: preprocess_company(row['git_company']), axis = 1)

In [ ]:
for column in ['git_name','full_name']:
    df[column] = df.apply(lambda row: preprocess(row[column]), axis = 1)

In [ ]:
def name_similarity(dataFrame, Col1, Col2):
    dataFrame['name_leven'] = dataFrame.apply(lambda row: ld(row[Col1], row[Col2]), axis = 1)
    dataFrame['name_dmetaphone'] = dataFrame.apply(lambda row: doublemetaphone(row[Col1], row[Col2]), axis = 1)
    dataFrame['name_jw'] = dataFrame.apply(lambda row: jaro_winkler(row[Col1], row[Col2]), axis = 1)
    dataFrame['name_fuzz'] = dataFrame.apply(lambda row: fuzz_sort(row[Col1], row[Col2]), axis = 1)
    return dataFrame

In [ ]:
df = name_similarity(df, 'git_name','full_name')

In [ ]:
df['login_fuzz'] = df.apply(lambda row: fuzz_sort(row['git_login'], row['full_name']), axis = 1)

df['login_jw'] = df.apply(lambda row: jaro_winkler(row['git_login'], row['full_name']), axis = 1)

df['location_fuzz_sort'] = df.apply(lambda row: fuzz_sort(row['git_location'], row['location']), axis = 1)

df['school_company'] = df.apply(lambda row: fuzz_sort(row['git_company'], row['edu_name']), axis = 1)

df['company_company'] = df.apply(lambda row: fuzz_sort(row['git_company'], row['exp_org']), axis = 1)

df['linkedin_gitblog'] = df.apply(lambda row: leven_list(row['linkedin_url'], row['git_blog']), axis = 1)

df['linkedin_gitweb'] = df.apply(lambda row: leven_list(row['linkedin_url'], row['git_websiteUrl']), axis = 1)

df['linkweb_gitblog'] = df.apply(lambda row: leven_list(row['website'], row['git_blog']), axis = 1)

df['linkweb_gitweb'] = df.apply(lambda row: leven_list(row['website'], row['git_websiteUrl']), axis = 1)

df['linkweb_github'] = df.apply(lambda row: leven_list(row['website'], row['github_url']), axis = 1)

In [ ]:
df[['linkedin_id','gid','count','name_leven','name_dmetaphone','name_jw','name_fuzz','login_fuzz','login_jw','location_fuzz_sort','school_company','company_company','linkedin_gitblog','linkedin_gitweb','linkweb_gitblog','linkweb_gitweb','linkweb_github']]

In [ ]:
from pyspark.sql.types import StructType, StructField, LongType

DFschema = StructType([
    StructField("linkedin_id", StringType()),
    StructField("gid", StringType()),
    StructField("count", LongType()),
    StructField("name_leven", IntegerType()),
    StructField("name_dmetaphone", FloatType()),
    StructField("name_jw", FloatType()),
    StructField("name_fuzz", IntegerType()),
    StructField("login_fuzz", IntegerType()),
    StructField("login_jw", FloatType()),
    StructField("location_fuzz_sort", IntegerType()),
    StructField("school_company", IntegerType()),
    StructField("company_company", IntegerType()),
    StructField("linkedin_gitblog", FloatType()),
    StructField("linkedin_gitweb", FloatType()),
    StructField("linkweb_gitblog", FloatType()),
    StructField("linkweb_gitweb", FloatType()),
    StructField("linkweb_github", FloatType())
   ])

In [ ]:
df = sqlContext.createDataFrame(df[['linkedin_id','gid','count','name_leven','name_dmetaphone','name_jw','name_fuzz','login_fuzz','login_jw','location_fuzz_sort','school_company','company_company','linkedin_gitblog','linkedin_gitweb','linkweb_gitblog','linkweb_gitweb','linkweb_github']], DFschema)

In [ ]:
feature = df.join(measureMapping, ['linkedin_id','gid'])

In [ ]:
feature.show()

In [ ]:
feature.persist()

In [ ]:
fill_values = {'bio_simi': -0.7676835240063782,'company_company': 13.027269554813302,'count': 69.97936515849672,'edu_exp_simi': -0.7674423412298906,'exp_desc_simi': -0.4794768606439826,'gitlang_simi': -0.36508407687661887,'linkedin_gitblog': 0.6704111164342388,'linkedin_gitweb': 0.6704111164342388,'linkweb_gitblog': 0.5065308678618619,'linkweb_github': 0.584223292437445,'linkweb_gitweb': 0.5065308678618619,'location_fuzz_sort': 34.35918477770764,'login_fuzz': 49.54616025581677,'login_jw': 0.638835025212626,'name_dmetaphone': 0.7347467665413555,'name_fuzz': 76.757491115495,'name_jw': 0.8106706649850334,'name_leven': 4.463146574747353,'pro_pub_desc_simi': -0.7593587783269232,'pro_pub_title_simi': -0.901583569164877,'school_company': 8.18176379679944, 'skill_simi': -0.29417421426755436, 'summary_simi': -0.9685668356171937}

In [ ]:
feature_f = feature.na.fill(fill_values).withColumnRenamed('match','label')

In [ ]:
from pyspark.ml import pipeline 
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer

assembler = VectorAssembler(inputCols=feature_f.columns[2:], outputCol="features")
feature_assem = assembler.transform(feature_f)

In [ ]:
# Or load the pre-trained Random Forest model from Azure

from pyspark.ml.classification import RandomForestClassificationModel
testRF= RandomForestClassificationModel.load("wasb://test-container@tfsmodelstorage.blob.core.windows.net/rfModel0914")

In [ ]:
rfPredict = testRF.transform(feature_assem)

In [ ]:
rfPredict.show()